In [ ]:
#
#File name: Prediction_and_bias_mitigation
#
#
# Programmed by Emeka Abakasanga
# Last revised:  February 2025
# Reference: Emeka Abakasanga*, Georgina Cosma*, Gyuchan Thomas Jun, Satheesh Gangadharan, Reza Kiani, 
# Danielle Fitt, Navjot Kaur, Francesco Zaccardi, Ashley Akbari, Rania Kousovista. 

# TITLE: Equitable Hospital Length of Stay Prediction for Patients with Learning Disabilities and 
# Multiple Long-term Conditions Using Machine Learning
# In: Frontiers in Digital Health 
# 
# Copyright (c) 2025 Emeka Abakasanga <E.Abakasanga@lboro.ac.uk>.

# https://www.frontiersin.org/journals/digital-health/articles/10.3389/fdgth.2025.1538793/abstract

In [ ]:
conda install -c conda-forge fairlearn
conda install -c conda-forge scikit-learn
conda install -c conda-forge xgboost
conda install seaborn -c conda-forge

In [ ]:
conda install conda-forge::pytorch-model-summary

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from dateutil.relativedelta import relativedelta
import datetime
import xgboost

pd.set_option("display.float_format", "{:.3f}".format)

In [ ]:
# changed from the original code as the ROC Curve function is not
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneOut
from xgboost import XGBClassifier
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.utils import Bunch
from sklearn.metrics import (
    balanced_accuracy_score,
    roc_auc_score,
    accuracy_score,
    recall_score,
    confusion_matrix,
    roc_auc_score,
    roc_curve,
    RocCurveDisplay)
from sklearn import set_config

set_config(display="diagram")
plt.style.use('seaborn-whitegrid')

In [3]:
from fairlearn.metrics import (
    MetricFrame,
    true_positive_rate,
    false_positive_rate,
    false_negative_rate,
    selection_rate,
    count,
    false_negative_rate_difference
)

from fairlearn.postprocessing import ThresholdOptimizer, plot_threshold_optimizer
from fairlearn.postprocessing._interpolated_thresholder import InterpolatedThresholder
from fairlearn.postprocessing._threshold_operation import ThresholdOperation
from fairlearn.reductions import ExponentiatedGradient, EqualizedOdds, TruePositiveRateParity, FalsePositiveRateParity

# **Overview of fairness in AI systems**

# **Fairlearn Bias Mitigation for Hospital Length Of Stay Prediction**


This Notebook builds on the following open source projects:

* **machine learning and data processing**: _scikit-learn_, _pandas_, _numpy_
* **plotting**: _seaborn_, _matplotlib_
* **AI fairness**: _Fairlearn_, _Model Card Toolkit_

### [Fairlearn](https://fairlearn.org)

Fairlearn is an open-source, community-driven project to help data scientists improve fairness of AI systems. It includes:

* A Python library for fairness assessment and improvement (fairness metrics, mitigation algorithms, plotting, etc.)

* Educational resources covering organizational and technical processes for unfairness mitigation (user guide, case studies, Jupyter notebooks, etc.)


# Healthcare Dataset and task

This study demonstrates the feasibility of applying ML models to predict LOS for patients with learning disabilities and multiple long term conditions, while addressing fairness through bias mitigation techniques. The findings highlight the potential for equitable healthcare predictions using EHR data, paving the way for improved clinical decision-making and resource management.

The study analyses hospitalisations of 9,618 patients with LD in Wales using electronic health records (EHR) from the SAIL Databank. Several ML models were applied to predict hospital LOS, incorporating demographics, medication history, lifestyle factors, and 39 long-term conditions and the optimal model was selected. To address fairness concerns, two bias mitigation techniques were applied on the optimal model: a post-processing threshold optimizer and an in-processing reductions method using an exponentiated gradient. These methods aimed to minimise performance discrepancies across ethnic groups while ensuring robust model performance.



**Convenience restriction**

* The datasets presented in the study are not readily available as all proposals to use SAIL data are subject to review by the independent IGRP. The anonymised individual-level data sources used in this study are available in the SAIL Databank at Swansea University, Swansea, UK, Before any data can be accessed, approval must be given by the IGRP. The IGRP gives careful consideration to each project to ensure proper and appropriate use of SAIL data. When access has been granted, it is gained through a privacy-protecting safe haven and remote access system referred to as the SAIL Gateway. SAIL has established an application process to be followed by anyone who would like to access data via SAIL at: https://www.saildatabank.com/application-process/.

## Fairness considerations

* _Which groups are most likely to be disproportionately negatively affected? Previous work suggests that patients from different ethnic groups might be affected.

* Because of the class imbalance, we will be measuring our performance via **balanced accuracy**. Another key performance consideration is the **false negative and false positive rate**.

* _What are the harms?_ The key harms here are allocation harms. In particular, false negatives, i.e., Recommending a patient to be discharged when they do need to stay in hospital admission.

* _How should we measure those harms?_


<a name="discussion-fairness-issues"></a>
## Discussion: Fairness-related harms

* How can we determine which type of harm is relevant in a particular scenario?
* What are ways to find out which (groups of) individuals are most likely to be disproportionately negatively affected?


## Load the dataset


We next load the dataset and review the meaning of its columns.


In [17]:
df = pd.read_csv("WP2_df_PEDW_TOTALEPISODES.csv")
df=df.drop('Unnamed: 0', axis=1)
df['DATE']=pd.to_datetime(df['DATE'])
df.columns

Index(['ALF_PE', 'DATE', 'DISCH_DT', 'EPI_STR_DT', 'EPI_END_DT', 'DIAG_NUM',
       'DIAG_CODE', 'DIAG_DESC', 'PROV_UNIT_CD', 'CONDITION', 'CODE_SYSTEM',
       'WOB', 'AGE_AT_INDEX_DATE', 'AGE_GRPS_AT_INDEX_DATE',
       'AGE_GRPS_AT_INDEX_DATE_DESC', 'DOD', 'GNDR_CD', 'GNDR_DESC',
       'LSOA2011_CD_AT_INDEX_DATE', 'WIMD2019_QUINTILE_AT_INDEX_DATE',
       'WIMD2019_QUINTILE_AT_INDEX_DATE_DESC',
       'TOWNSEND2011_QUINTILE_AT_INDEX_DATE',
       'TOWNSEND2011_QUINTILE_AT_INDEX_DATE_DESC',
       'ETHN_EC_ONS_DATE_LATEST_CODE', 'ETHN_EC_ONS_DATE_LATEST_DESC',
       'ETHN_EC_NER_DATE_LATEST_CODE', 'ETHN_EC_NER_DATE_LATEST_DESC',
       'AGE_AT_DATE', 'AGEGRP_AT_DATE', 'MORTALITY', 'COM_SEQ',
       'COMORBIDITY_COUNT', 'AUTISM', 'BEHAVIOR_DISORDER', 'ALCOHOL_HISTORY',
       'PresenceAlcRec', 'PresenceALCchange', 'SMOKING_HISTORY',
       'PresenceSMOKchange', 'PresenceSmokRec', 'MEDICATIONS',
       'PresenceMEDIchange', 'PHYSICAL', 'PresencePHYSRec',
       'PresencePHYSchange', 

In [18]:
#ADJUST COHORT TO ONLY TAKE IN ADMISSIONS FROM 2000
dte=pd.to_datetime('2000-01-01 00:00:00')
df=df.query('DATE>=@dte')

In [19]:
#OBTAIN TIMES CONDITIONS WERE TREATED DURING UNIQUE ADMISSIONS
df.CONDITION.value_counts().to_csv('Cond_Total.csv')

In [ ]:
# NUMBE ROF OF UNIQUE PATIENTS
df.ALF_PE.nunique()

MORTALITY DISTRIBUTION

In [ ]:
df.query('HOSP_MORTALITY==1').drop_duplicates(subset=['ALF_PE'], ignore_index=True,  keep='last').shape[0]

In [22]:
MALE_IHM=df.query("HOSP_MORTALITY==1 and GNDR_DESC=='MALE'").drop_duplicates(subset=['ALF_PE'], ignore_index=True,  keep='last')

In [23]:
FEMALE_IHM=df.query("HOSP_MORTALITY==1 and GNDR_DESC=='FEMALE'").drop_duplicates(subset=['ALF_PE'], ignore_index=True,  keep='last')

In [ ]:
MALE_IHM['WIMD2019_QUINTILE_AT_INDEX_DATE'].value_counts()

In [ ]:
FEMALE_IHM['WIMD2019_QUINTILE_AT_INDEX_DATE'].value_counts()

QUERY DATA TO ONLY INCLUDE INFORMATION OF FIRST 24 HOURS OF ADMISSION

In [26]:
df=df.query('EPI_STR_DT<=DATE_AFTER_ADMIS').reset_index(drop=True) #OBTAIN ONLY DATA WITHIN 24 HOURS OF ADMISSION
df=df.sort_values(by=['ALF_PE','DATE', 'DISCH_DT','EPI_STR_DT'], ignore_index=True)
df=df.drop_duplicates(subset=['ALF_PE','DATE', 'DISCH_DT','CONDITION'], ignore_index=True)#DROP DUPLICATE CONDITIONS WITHIN AN ADMISSION

# DEFINE COLUMN FOR TARGET VARIABLE
df['LOSClass']=0
df.loc[df['LOS']>=4, 'LOSClass']=1

In [29]:
# MAKE UNIQUE GROUPINGS OF ALL CONDITIONS IN THE FIRST 24 HOURS OF EACH UNIQUE ADMISSION
df.sort_values(by=['ALF_PE','DATE', 'DISCH_DT'], ignore_index=True)
DF_CHECK=df
DF_CHECK=DF_CHECK.sort_values(by=['ALF_PE','DATE', 'DISCH_DT'], ignore_index=True)
DF_CHECK=DF_CHECK.drop_duplicates(subset=['ALF_PE','DATE', 'DISCH_DT'], ignore_index=True)
s=df.groupby(['ALF_PE','DATE', 'DISCH_DT'],group_keys=False)['CONDITION'].unique().to_frame('COND').reset_index()

DF_CHECK['COND']=s['COND']
df=pd.merge(df,DF_CHECK[['ALF_PE','DATE','DISCH_DT','COND']])

In [31]:
#OBTAIN TOTAL COMORBIDITY AS AT 24 HRS OF ADMISSION
s= df.groupby(['ALF_PE','DATE', 'DISCH_DT'],group_keys=False)['COMORBIDITY_COUNT'].max().to_frame('TOTAL_COMORBIDITY').reset_index()

df= pd.merge(df,s[['ALF_PE','DATE','DISCH_DT','TOTAL_COMORBIDITY']])

In [32]:
df=df.drop_duplicates(subset=['ALF_PE','DATE', 'DISCH_DT'], ignore_index=True,  keep='last')

In [ ]:
#sELECT THE REQUIRED  VARIABLES 
df = df[['ALF_PE','DISCH_DT',
         'DATE',
         'CONDITION',
         'COND',
         'GNDR_DESC',
         'WIMD2019_QUINTILE_AT_INDEX_DATE',
         'TOWNSEND2011_QUINTILE_AT_INDEX_DATE',
         'ETHN_EC_ONS_DATE_LATEST_DESC',
         'LOS',
         'COM_SEQ', 'COMORBIDITY_COUNT', 'AUTISM',
         'ALCOHOL_HISTORY',
         'SMOKING_HISTORY',
         'MEDICATIONS', 'PHYSICAL', 
         'BMI', 'AGEGRP_AT_ADMIS_DT',
          'NUM_PRVEPISODES_1YR', 'NUM_PRVCOMORBID_1YR', 'NUM_PRVCOMORBID_3YR',
         'NUM_PRVEPISODES_3YR','NUMEPISODES_24HRS',
         'NUMCOMORBIDITIES_24HRS', 'NUM_PRVADMISSION_1YR_COND',
         'NUM_PRVADMISSION_3YR_COND','NUM_PRVADMISSION_1YR','NUM_PRVADMISSION_3YR',
         'NUM_PRVHOSPITAL_DAYS_1YR', 'NUM_PRVHOSPITAL_DAYS_3YR',
         'LOSClass', 'LOSClass2','TOTAL_COMORBIDITY']]


df.rename(columns={'ETHN_EC_ONS_DATE_LATEST_DESC': 'Ethnicity'}, inplace=True)

#MERGE 'MIXED' AND 'OTHER' ETHNICITY GROUPS
df['Ethnicity']=df['Ethnicity'].replace({"Mixed":"Other"})

#RENAME AGEGROUP
df['AGEGRP_AT_ADMIS_DT']=df['AGEGRP_AT_ADMIS_DT'].replace({"<20":"0-20"})



# REPLACE MISSING CATEGORICAL VALUES WITH 'UNKNOWN' FOR ETHNICITY, GENDER AND WIMD 
df['Ethnicity'].fillna("Unknown", inplace=True)
df['WIMD2019_QUINTILE_AT_INDEX_DATE'].fillna("Unknown", inplace=True)
df['GNDR_DESC'].fillna("Unknown", inplace=True)

#FILTER ONLY FEMALE PATIENTS
df_female=df.query('GNDR_DESC=="FEMALE"')


df_female=df_female.drop(['DATE', 'DISCH_DT'], axis=1).reset_index(drop=True) 
df1=df1.drop(['DATE', 'DISCH_DT'], axis=1).reset_index(drop=True)

In [ ]:
#NUMBER OF UNIQUE ADMISSIONS
df_female.shape[0]

In [ ]:
#OBTAIN NORMALISED CONUTS OF LOSClass
df_female.LOSClass.value_counts(normalize=True)

In [ ]:
#Ethbnicity value counts
df_female.Ethnicity.value_counts()

In [ ]:
# number of unique conditions throughout all admissions 
df_female.CONDITION.nunique()

In [42]:
# df_female.to_csv('df_ML_female.csv')

In [ ]:
# CATEGORTICAL VARIABLES
categorical_values = {}
for col in df_female:
  if col not in {'ALF_PE','COMORBIDITY_COUNT', 'LOS', 'NUM_PRVADMISSION_1YR', 'NUM_PRVADMISSION_1YR_COND', 'CONDITION',
       'NUM_PRVADMISSION_3YR', 'NUM_PRVADMISSION_3YR_COND','AGE_AT_ADMIS_DT',
       'NUM_PRVHOSPITAL_DAYS_1YR', 'NUM_PRVHOSPITAL_DAYS_3YR','COM_SEQ',
       'TOTAL_UNIQ_HOSPITALIZATIONS_AT_ENDDATE', 'NUM_PRVEPISODES_1YR', 'NUM_PRVCOMORBID_1YR',
       'NUM_PRVCOMORBID_3YR','NUMEPISODES_24HRS', 'NUMCOMORBIDITIES_24HRS','NUM_PRVEPISODES_3YR','NUM_ADMISSION_COMORBIDITIES','TOTAL_COMORBIDITY'}:
    categorical_values[col] = pd.Series(df_female[col].value_counts().index.values)
categorical_values_df = pd.DataFrame(categorical_values)
categorical_values_df.T

We mark all categorical features: 

In [44]:
categorical_features = [
        'CONDITION',
         'GNDR_DESC',
         'WIMD2019_QUINTILE_AT_INDEX_DATE',
         'TOWNSEND2011_QUINTILE_AT_INDEX_DATE',
         'Ethnicity',
         'AUTISM',
         'ALCOHOL_HISTORY',
         'SMOKING_HISTORY',
         'MEDICATIONS', 
         'PHYSICAL',
         'BMI', 
         'AGEGRP_AT_ADMIS_DT',
         'LOSClass',
         'LOSClass2'
]

In [45]:
# set category type categorical variables
for col_name in categorical_features:
    df_female[col_name] = df_female[col_name].astype("category")

In [51]:
# OBTAIN THE DISTRIBUTION OF LOSClass ACROSS SOME CATEGORICAL VARIABLES
cat_list=[
       'WIMD2019_QUINTILE_AT_INDEX_DATE',
       'AUTISM', 'ALCOHOL_HISTORY', 'SMOKING_HISTORY',
       'MEDICATIONS', 'PHYSICAL', 'BMI', 'AGEGRP_AT_ADMIS_DT'
       ]

a=df1.groupby('Ethnicity')['LOSClass'].agg(['count','mean']).reset_index()

for ii in range(len(cat_list)):
    b=df1.groupby(cat_list[ii])['LOSClass'].agg(['count','mean']).reset_index()
    a=pd.concat([a,b], axis=0)
    
a.to_csv('Group_StatFemale.csv')

In [ ]:
df1.query("LOSClass==1").Ethnicity.value_counts()

**Ethnicity statistics**

In [61]:
# Ethnicity based on number of admissions
df1['Ethnicity'].value_counts().to_csv("ethnicity_row_based_fem.csv")

In [62]:
# Ethnicity based on number of users
df1[['ALF_PE', 'Ethnicity']].drop_duplicates()['Ethnicity'].value_counts().to_csv("ethnicity_patient_based_fem.csv")

In [ ]:
# Plot Ethnicity based on rows with log scale
fig = plt.figure()
sns.countplot(x='Ethnicity', data=df1)
plt.xlabel("Ethnicity")
plt.ylabel("Count (log scale)")
plt.yscale('log')
plt.show
fig.savefig("ethnicity_row_based_log_fem.png", dpi=300, bbox_inches= "tight")

In [ ]:
# Plot Ethnicity without log scale across admissions
fig = plt.figure()
sns.countplot(x='Ethnicity', data=df1)
plt.xlabel("Ethnicity")
plt.ylabel("Count")
plt.show
fig.savefig("ethnicity_row_based_fem.png", dpi=300, bbox_inches= "tight")

In [ ]:
# with log scale unique Patient ethnicity count plot 
df_ethn_user = df1[['ALF_PE', 'Ethnicity']].drop_duplicates()
fig = plt.figure()
sns.countplot(x='Ethnicity', data=df_ethn_user)
plt.xlabel("Ethnicity")
plt.ylabel("Count (log scale)")
plt.yscale('log')
plt.show
fig.savefig("ethnicity_user_based_log_fem.png", dpi=300, bbox_inches= "tight")

In [ ]:
# without log scale unique Patient ethnicity count plot
fig = plt.figure()
sns.countplot(x='Ethnicity', data=df_ethn_user)
plt.xlabel("Ethnicity")
plt.ylabel("Count")
plt.show
fig.savefig("ethnicity_user_based_fem.png", dpi=300, bbox_inches= "tight")

In [68]:
# per admissions
df1['Ethnicity'].value_counts(normalize=True).to_csv("ethn_normal_per_row_fem.csv")

In [69]:
# per patient
df1[['ALF_PE', 'Ethnicity']].drop_duplicates()['Ethnicity'].value_counts(normalize=True).to_csv("ethn_normal_per_user_fem.csv")

In [70]:
df1.groupby('Ethnicity')['LOSClass'].agg(['count','mean']).reset_index().to_csv('row_ethn_LOS_fem.csv')

**WIMD statistics**

In [72]:
# Ethnicity based on number of rows
df1['WIMD2019_QUINTILE_AT_INDEX_DATE'].value_counts().to_csv("wimd_row_based_fem.csv")

In [ ]:
df1['WIMD2019_QUINTILE_AT_INDEX_DATE'].value_counts()

In [74]:
# WIMD based on number of users
df1[['ALF_PE', 'WIMD2019_QUINTILE_AT_INDEX_DATE']].drop_duplicates()['WIMD2019_QUINTILE_AT_INDEX_DATE'].value_counts().to_csv("wimd_user_based_fem.csv")

In [ ]:
# WIMD based on rows with normal scale
fig = plt.figure()
sns.countplot(x='WIMD2019_QUINTILE_AT_INDEX_DATE', data=df1)
plt.xlabel("WIMD 2019 - Quintile")
plt.ylabel("Count")
plt.show
fig.savefig("wimd_row_based_fem.png", dpi=300, bbox_inches= "tight")

In [ ]:
# WIMD without log scale per user
df_wimd_user = df1[['ALF_PE', 'WIMD2019_QUINTILE_AT_INDEX_DATE']].drop_duplicates()
fig = plt.figure()
sns.countplot(x='WIMD2019_QUINTILE_AT_INDEX_DATE', data=df_wimd_user)
plt.xlabel("WIMD 2019 - Quintile")
plt.ylabel("Count")
plt.show
fig.savefig("wimd_user_based_fem.png", dpi=300, bbox_inches= "tight")


In [77]:
# WIMD per row
df1['WIMD2019_QUINTILE_AT_INDEX_DATE'].value_counts(normalize=True).to_csv("wimd_normal_per_row_fem.csv")

In [78]:
# WIMD per user
df1[['ALF_PE', 'WIMD2019_QUINTILE_AT_INDEX_DATE']].drop_duplicates()['WIMD2019_QUINTILE_AT_INDEX_DATE'].value_counts(normalize=True).to_csv("wimd_normal_per_user_fem.csv")

In [79]:
df1.groupby('WIMD2019_QUINTILE_AT_INDEX_DATE')['LOSClass'].agg(['mean']).reset_index().to_csv('row_wimd_LOS_fem.csv')

**Age**

In [80]:
# df_age = df1[['ALF_PE', 'AGEGRP_AT_ADMIS_DT']].drop_duplicates(keep='last')
df_age = df1.drop_duplicates(subset=['ALF_PE'],keep='last')

In [ ]:
df_age.shape

In [85]:
df_age["AGEGRP_AT_ADMIS_DT"].value_counts().to_csv("user_age_fem.csv")

In [86]:
df_age["AGEGRP_AT_ADMIS_DT"].value_counts(normalize=True).to_csv("user_age_norm_fem.csv")

In [87]:
df1["AGEGRP_AT_ADMIS_DT"].value_counts().to_csv("row_age_fem.csv")

In [88]:
df1["AGEGRP_AT_ADMIS_DT"].value_counts(normalize=True).to_csv("row_age_norm_fem.csv")

In [ ]:
fig = plt.figure(figsize=(20,7))
category_order = sorted(df['AGEGRP_AT_ADMIS_DT'].unique())
sns.countplot(x='AGEGRP_AT_ADMIS_DT', data=df_age, order = category_order)
plt.xlabel("Age group")
plt.ylabel("Count")
plt.show()
fig.savefig("user_age_fem.png", dpi=300, bbox_inches = "tight")

In [ ]:
fig = plt.figure(figsize=(20,7))
category_order = sorted(df['AGEGRP_AT_ADMIS_DT'].unique())
sns.countplot(x='AGEGRP_AT_ADMIS_DT', data=df1, order = category_order)
plt.xlabel("Age group")
plt.ylabel("Count")
plt.show()
fig.savefig("row_age_fem.png", dpi=300, bbox_inches = "tight")

In [91]:
#obtain LOSClass distribution across each age group in unique admissions
df1.groupby('AGEGRP_AT_ADMIS_DT')['LOSClass'].agg(['mean']).reset_index().to_csv('row_age_LOS_fem.csv')

In [ ]:
df1.LOSClass.describe()

LOS CLASSIFICATION
CASES WITH LOS > THRESHOLD (4 DAYS) WILL BE CLASSIFIED AS 1; CASES WITH LOS <= THRESHOLD (4 DAYS) WILL BE CLASSIFIED AS 0

In [ ]:
df_female["LOSClass"].value_counts(normalize=True)

In [134]:
df_female["LOSClass"].value_counts().to_csv("row_LOS_fem.csv")

In [53]:
df_female.columns

Index(['ALF_PE', 'CONDITION', 'GNDR_DESC', 'WIMD2019_QUINTILE_AT_INDEX_DATE',
       'TOWNSEND2011_QUINTILE_AT_INDEX_DATE', 'Ethnicity', 'LOS', 'COM_SEQ',
       'COMORBIDITY_COUNT', 'AUTISM', 'ALCOHOL_HISTORY', 'SMOKING_HISTORY',
       'MEDICATIONS', 'PHYSICAL', 'BMI', 'AGEGRP_AT_ADMIS_DT',
       'NUM_PRVADMISSION_1YR', 'NUM_PRVEPISODES_1YR', 'NUM_PRVCOMORBID_1YR',
       'NUM_PRVCOMORBID_3YR', 'NUM_PRVEPISODES_3YR', 'NUMEPISODES_24HRS',
       'NUMCOMORBIDITIES_24HRS', 'NUM_PRVADMISSION_1YR_COND',
       'NUM_PRVADMISSION_3YR', 'NUM_PRVADMISSION_3YR_COND',
       'NUM_PRVHOSPITAL_DAYS_1YR', 'NUM_PRVHOSPITAL_DAYS_3YR', 'LOSClass',
       'LOSClass2'],
      dtype='object')

    POINT PLOTS TO SHOW THE RELATIONSHIP BETWEEN NUMERICAL VARIABLES AND THE LOS CLASSES

In [ ]:
fig = plt.figure()
sns.pointplot(y="TOTAL_COMORBIDITY", x="LOSClass", data=df_female, ci=95, join=False);
plt.xlabel("LOS$\geq$4 DAYS")
plt.ylabel("TOTAL_COMORBIDITY")
plt.show
fig.savefig("TOTAL_COMORBIDITYfemale.png", dpi=300, bbox_inches= "tight")

In [ ]:
fig = plt.figure()
sns.pointplot(y="NUM_PRVHOSPITAL_DAYS_3YR", x="LOSClass", data=df_female, ci=95, join=False);
plt.xlabel("LOS$\geq$4 DAYS")
plt.ylabel("NUM_PRVHOSPITAL_DAYS_3YR")
plt.show
fig.savefig("NUM_PRVHOSPITAL_DAYS_3YRfemale.png", dpi=300, bbox_inches= "tight")

In [ ]:
fig = plt.figure()
sns.pointplot(y="NUM_PRVHOSPITAL_DAYS_1YR", x="LOSClass", data=df_female, ci=95, join=False);
plt.xlabel("LOS$\geq$4 DAYS")
plt.ylabel("NUM_PRVHOSPITAL_DAYS_1YR")
plt.show
fig.savefig("NUM_PRVHOSPITAL_DAYS_1YRfemale.png", dpi=300, bbox_inches= "tight")

In [ ]:
fig = plt.figure()
sns.pointplot(y="NUM_PRVCOMORBID_1YR", x="LOSClass", data=df_female, ci=95, join=False);
plt.xlabel("LOS$\geq$4 DAYS")
plt.ylabel("NUM_PRVCOMORBID_1YR")
plt.show
fig.savefig("NUM_PRVCOMORBID_1YRfemale.png", dpi=300, bbox_inches= "tight")

In [ ]:
fig = plt.figure()
sns.pointplot(y="NUM_PRVCOMORBID_3YR", x="LOSClass", data=df_female, ci=95, join=False);
plt.xlabel("LOS$\geq$4 DAYS")
plt.ylabel("NUM_PRVCOMORBID_3YR")
plt.show
fig.savefig("NUM_PRVCOMORBID_3YRfemale.png", dpi=300, bbox_inches= "tight")

In [ ]:
fig = plt.figure()
sns.pointplot(y="NUM_PRVEPISODES_1YR", x="LOSClass", data=df_female, ci=95, join=False);
plt.xlabel("LOS$\geq$4 DAYS")
plt.ylabel("NUM_PRVEPISODES_1YR")
plt.show
fig.savefig("NUM_PRVEPISODES_1YRfemale.png", dpi=300, bbox_inches= "tight")

In [ ]:
fig = plt.figure()
sns.pointplot(y="NUM_PRVEPISODES_3YR", x="LOSClass", data=df_female, ci=95, join=False);
plt.xlabel("LOS$\geq$4 DAYS")
plt.ylabel("NUM_PRVEPISODES_3YR")
plt.show
fig.savefig("NUM_PRVEPISODES_3YRfemale.png", dpi=300, bbox_inches= "tight")

In [ ]:
fig = plt.figure()
sns.pointplot(y="NUMEPISODES_24HRS", x="LOSClass", data=df_female, ci=95, join=False);
plt.xlabel("LOS$\geq$4 DAYS")
plt.ylabel("NUMEPISODES_24HRS")
plt.show
fig.savefig("NUMEPISODES_24HRSfemale.png", dpi=300, bbox_inches= "tight")

In [ ]:
fig = plt.figure()
sns.pointplot(y="NUMCOMORBIDITIES_24HRS", x="LOSClass", data=df_female, ci=95, join=False);
plt.xlabel("LOS$\geq$4 DAYS")
plt.ylabel("NUMCOMORBIDITIES_24HRS")
plt.show
fig.savefig("NUMCOMORBIDITIES_24HRSfemale.png", dpi=300, bbox_inches= "tight")

In [ ]:
fig = plt.figure()
sns.pointplot(y="NUM_PRVADMISSION_1YR", x="LOSClass", data=df_female, ci=95, join=False);
plt.xlabel("LOS$\geq$4 DAYS")
plt.ylabel("NUM_PRVADMISSION_1YR")
plt.show
fig.savefig("NUM_PRVADMISSION_1YRfemale.png", dpi=300, bbox_inches= "tight")

In [ ]:
fig = plt.figure()
sns.pointplot(y="NUM_PRVADMISSION_3YR", x="LOSClass", data=df_female, ci=95, join=False);
plt.xlabel("LOS$\geq$4 DAYS")
plt.ylabel("NUM_PRVADMISSION_3YR")
plt.show
fig.savefig("NUM_PRVADMISSION_3YRfemale.png", dpi=300, bbox_inches= "tight")

In [135]:
df_female["LOSClass"].value_counts(normalize=True).to_csv("row_LOS_norm_fem.csv")

VALIDITY OF NUM_PRVADMISSION_1YR IN OUR PREDICTION OF LOS FOR 7 DAYS THRESHOLD

In [351]:
df_female.groupby(['LOSClass2', 'Ethnicity'])['NUM_PRVADMISSION_1YR'].agg(['mean', 'std']).reset_index().to_csv("prev_adm_LOS2_ethn_stat.csv")

In [352]:
# PREVADMISSIONS _READMIT30DAYS STAT_GENDER Stat
df_female.groupby(['LOSClass2', 'GNDR_DESC'])['NUM_PRVADMISSION_1YR'].agg(['mean', 'std']).reset_index().to_csv("prev_adm_LOS2_gender_stat.csv")

In [353]:
df_female.groupby(['LOSClass2', 'WIMD2019_QUINTILE_AT_INDEX_DATE'])['NUM_PRVADMISSION_1YR'].agg(['mean', 'std']).reset_index().to_csv("prev_adm_LOS2_wimd_stat.csv")

In [354]:
df_female.groupby(['LOSClass2'])['NUM_PRVADMISSION_1YR'].agg(['mean', 'std']).reset_index().to_csv("prev_adm_LOS2.csv")

In [355]:
df_female.columns

Index(['ALF_PE', 'CONDITION', 'GNDR_DESC', 'WIMD2019_QUINTILE_AT_INDEX_DATE',
       'TOWNSEND2011_QUINTILE_AT_INDEX_DATE', 'Ethnicity', 'LOS', 'COM_SEQ',
       'COMORBIDITY_COUNT', 'AUTISM', 'ALCOHOL_HISTORY', 'SMOKING_HISTORY',
       'MEDICATIONS', 'PHYSICAL', 'BMI', 'AGEGRP_AT_ADMIS_DT',
       'NUM_PRVADMISSION_1YR', 'NUM_PRVEPISODES_1YR', 'NUM_PRVCOMORBID_1YR',
       'NUM_PRVCOMORBID_3YR', 'NUM_PRVEPISODES_3YR', 'NUMEPISODES_24HRS',
       'NUMCOMORBIDITIES_24HRS', 'NUM_PRVADMISSION_1YR_COND',
       'NUM_PRVADMISSION_3YR', 'NUM_PRVADMISSION_3YR_COND',
       'NUM_PRVHOSPITAL_DAYS_1YR', 'NUM_PRVHOSPITAL_DAYS_3YR', 'LOSClass',
       'LOSClass2'],
      dtype='object')

In [ ]:
# previous admissions- readmission - ethnicity figure (dodge)
fig = plt.figure()
sns.pointplot(y="NUM_PRVADMISSION_1YR", x="LOSClass2", hue="Ethnicity",
             data=df_female, ci=95, join=False, dodge=True);
plt.xlabel("LOS_7DAYS")
plt.ylabel("Number of Admissions in Previous year")
plt.show
fig.savefig("prev_adm__LOS2_ethn_dodge.png", dpi=300, bbox_inches= "tight")

In [ ]:
# PREVADMISSIONS _READMIT30DAYS STAT_GENDER figure normal
fig = plt.figure()
aa = sns.pointplot(y="NUM_PRVADMISSION_1YR", x="LOSClass2", hue="GNDR_DESC",
             data=df_female, ci=95, join=False, dodge=True);

plt.xlabel("LOS_7DAYS")
plt.ylabel("Number of Admissions in Previous year")
sns.move_legend(aa, loc='upper left', bbox_to_anchor=(1,1))
plt.legend(title="Gender")
plt.show
fig.savefig("prev_adm__LOS2_gender.png", dpi=300, bbox_inches= "tight")

In [ ]:
# PREVADMISSIONS _READMIT30DAYS WIMD2019_QUINTILE_AT_INDEX_DATE Figure (dodge)
fig = plt.figure()
aa = sns.pointplot(y="NUM_PRVADMISSION_1YR", x="LOSClass2", hue="WIMD2019_QUINTILE_AT_INDEX_DATE",
             data=df_female, ci=95, dodge=True,  join=False);
plt.xlabel("LOS_7DAYS")
plt.ylabel("Number of Admissions in Previous year")
sns.move_legend(aa, 'upper left', bbox_to_anchor=(1,1))
plt.legend(title="WIMD 2019 - Quintile")
plt.show
fig.savefig("prev_adm__LOS2_wimd_dodge.png", dpi=300, bbox_inches= "tight")

In [ ]:
fig = plt.figure()
sns.pointplot(y="NUM_PRVADMISSION_1YR", x="LOSClass2", data=df_female, ci=95, join=False);
plt.xlabel("LOS_7DAYS")
plt.ylabel("Number of Admissions in Previous year")
plt.show
fig.savefig("prev_adm_LOS2.png", dpi=300, bbox_inches= "tight")

VALIDITY OF NUM_PRVADMISSION_1YR IN OUR PREDICTION OF LOS FOR 4 DAYS THRESHOLD

In [303]:
df_female.groupby(['LOSClass', 'Ethnicity'])['NUM_PRVADMISSION_1YR'].agg(['mean', 'std']).reset_index().to_csv("prev_adm_LOS_ethn_stat.csv")

In [304]:
# PREVADMISSIONS _READMIT30DAYS STAT_GENDER Stat
df_female.groupby(['LOSClass', 'GNDR_DESC'])['NUM_PRVADMISSION_1YR'].agg(['mean', 'std']).reset_index().to_csv("prev_adm_LOS_gender_stat.csv")

In [305]:
df_female.groupby(['LOSClass', 'WIMD2019_QUINTILE_AT_INDEX_DATE'])['NUM_PRVADMISSION_1YR'].agg(['mean', 'std']).reset_index().to_csv("prev_adm_LOS_wimd_stat.csv")

In [306]:
df_female.groupby(['LOSClass'])['NUM_PRVADMISSION_1YR'].agg(['mean', 'std']).reset_index().to_csv("prev_adm_LOS.csv")

In [307]:
df_female.columns

Index(['ALF_PE', 'CONDITION', 'GNDR_DESC', 'WIMD2019_QUINTILE_AT_INDEX_DATE',
       'TOWNSEND2011_QUINTILE_AT_INDEX_DATE', 'Ethnicity', 'LOS', 'COM_SEQ',
       'COMORBIDITY_COUNT', 'AUTISM', 'ALCOHOL_HISTORY', 'SMOKING_HISTORY',
       'MEDICATIONS', 'PHYSICAL', 'BMI', 'AGEGRP_AT_ADMIS_DT',
       'NUM_PRVADMISSION_1YR', 'NUM_PRVEPISODES_1YR', 'NUM_PRVCOMORBID_1YR',
       'NUM_PRVCOMORBID_3YR', 'NUM_PRVEPISODES_3YR', 'NUMEPISODES_24HRS',
       'NUMCOMORBIDITIES_24HRS', 'NUM_PRVADMISSION_1YR_COND',
       'NUM_PRVADMISSION_3YR', 'NUM_PRVADMISSION_3YR_COND',
       'NUM_PRVHOSPITAL_DAYS_1YR', 'NUM_PRVHOSPITAL_DAYS_3YR', 'LOSClass',
       'LOSClass2'],
      dtype='object')

In [ ]:
# previous admissions- readmission - ethnicity figure (dodge)
fig = plt.figure()
sns.pointplot(y="NUM_PRVADMISSION_1YR", x="LOSClass", hue="Ethnicity",
             data=df_female, ci=95, join=False, dodge=True);
plt.xlabel("LOS_4DAYS")
plt.ylabel("Number of Admissions in Previous year")
plt.show
fig.savefig("prev_adm__LOS_ethn_dodge.png", dpi=300, bbox_inches= "tight")

In [ ]:
# PREVADMISSIONS _READMIT30DAYS STAT_GENDER figure normal
fig = plt.figure()
aa = sns.pointplot(y="NUM_PRVADMISSION_1YR", x="LOSClass", hue="GNDR_DESC",
             data=df_female, ci=95, join=False, dodge=True);

plt.xlabel("LOS_4DAYS")
plt.ylabel("Number of Admissions in Previous year")
sns.move_legend(aa, loc='upper left', bbox_to_anchor=(1,1))
plt.legend(title="Gender")
plt.show
fig.savefig("prev_adm__LOS_gender.png", dpi=300, bbox_inches= "tight")

In [ ]:
# PREVADMISSIONS _READMIT30DAYS WIMD2019_QUINTILE_AT_INDEX_DATE Figure (dodge)
fig = plt.figure()
aa = sns.pointplot(y="NUM_PRVADMISSION_1YR", x="LOSClass", hue="WIMD2019_QUINTILE_AT_INDEX_DATE",
             data=df_female, ci=95, dodge=True,  join=False);
plt.xlabel("LOS_4DAYS")
plt.ylabel("Number of Admissions in Previous year")
sns.move_legend(aa, 'upper left', bbox_to_anchor=(1,1))
plt.legend(title="WIMD 2019 - Quintile")
plt.show
fig.savefig("prev_adm__LOS_wimd_dodge.png", dpi=300, bbox_inches= "tight")

In [ ]:
fig = plt.figure()
sns.pointplot(y="NUM_PRVADMISSION_3YR_COND", x="LOSClass", data=df_female, ci=95, join=False);
plt.xlabel("LOS_4DAYS")
plt.ylabel("Number of Admissions in Previous 3 years")
plt.show
#fig.savefig("prev_adm_LOS.png", dpi=300, bbox_inches= "tight")

In [ ]:
fig = plt.figure()
sns.pointplot(y="NUM_PRVADMISSION_1YR", x="LOSClass", data=df_female, ci=95, join=False);
plt.xlabel("LOS_4DAYS")
plt.ylabel("Number of Admissions in Previous year")
plt.show
fig.savefig("prev_adm_LOS.png", dpi=300, bbox_inches= "tight")

VALIDITY OF NUM_PRVHOSPITAL_DAYS_1YR IN OUR PREDICTION OF LOS FOR 7 DAYS THRESHOLD

In [369]:
df_female.groupby(['LOSClass2', 'Ethnicity'])['NUM_PRVHOSPITAL_DAYS_1YR'].agg(['mean', 'std']).reset_index().to_csv("prev_days_LOS2_ethn_stat.csv")

In [370]:
# PREVADMISSIONS _READMIT30DAYS STAT_GENDER Stat
df_female.groupby(['LOSClass2', 'GNDR_DESC'])['NUM_PRVHOSPITAL_DAYS_1YR'].agg(['mean', 'std']).reset_index().to_csv("prev_days_LOS2_gender_stat.csv")

In [371]:
df_female.groupby(['LOSClass2', 'WIMD2019_QUINTILE_AT_INDEX_DATE'])['NUM_PRVHOSPITAL_DAYS_1YR'].agg(['mean', 'std']).reset_index().to_csv("prev_days_LOS2_wimd_stat.csv")

In [372]:
df_female.groupby(['LOSClass2'])['NUM_PRVHOSPITAL_DAYS_1YR'].agg(['mean', 'std']).reset_index().to_csv("prev_days_LOS2.csv")

In [373]:
df_female.columns

Index(['ALF_PE', 'CONDITION', 'GNDR_DESC', 'WIMD2019_QUINTILE_AT_INDEX_DATE',
       'TOWNSEND2011_QUINTILE_AT_INDEX_DATE', 'Ethnicity', 'LOS', 'COM_SEQ',
       'COMORBIDITY_COUNT', 'AUTISM', 'ALCOHOL_HISTORY', 'SMOKING_HISTORY',
       'MEDICATIONS', 'PHYSICAL', 'BMI', 'AGEGRP_AT_ADMIS_DT',
       'NUM_PRVADMISSION_1YR', 'NUM_PRVEPISODES_1YR', 'NUM_PRVCOMORBID_1YR',
       'NUM_PRVCOMORBID_3YR', 'NUM_PRVEPISODES_3YR', 'NUMEPISODES_24HRS',
       'NUMCOMORBIDITIES_24HRS', 'NUM_PRVADMISSION_1YR_COND',
       'NUM_PRVADMISSION_3YR', 'NUM_PRVADMISSION_3YR_COND',
       'NUM_PRVHOSPITAL_DAYS_1YR', 'NUM_PRVHOSPITAL_DAYS_3YR', 'LOSClass',
       'LOSClass2'],
      dtype='object')

In [ ]:
# previous admissions- readmission - ethnicity figure (dodge)
fig = plt.figure()
sns.pointplot(y="NUM_PRVHOSPITAL_DAYS_1YR", x="LOSClass2", hue="Ethnicity",
             data=df_female, ci=95, join=False, dodge=True);
plt.xlabel("LOS_7DAYS")
plt.ylabel("Number of Hospital Days in Previous year")
plt.show
fig.savefig("prev_days__LOS2_ethn_dodge.png", dpi=300, bbox_inches= "tight")

In [ ]:
# PREVADMISSIONS _READMIT30DAYS WIMD2019_QUINTILE_AT_INDEX_DATE Figure (dodge)
fig = plt.figure()
aa = sns.pointplot(y="NUM_PRVHOSPITAL_DAYS_1YR", x="LOSClass2", hue="WIMD2019_QUINTILE_AT_INDEX_DATE",
             data=df_female, ci=95, dodge=True,  join=False);
plt.xlabel("LOS_7DAYS")
plt.ylabel("Number of Hospital Days in Previous year")
sns.move_legend(aa, 'upper left', bbox_to_anchor=(1,1))
plt.legend(title="WIMD 2019 - Quintile")
plt.show
fig.savefig("prev_days__LOS2_wimd_dodge.png", dpi=300, bbox_inches= "tight")

In [ ]:
fig = plt.figure()
sns.pointplot(y="NUM_PRVHOSPITAL_DAYS_1YR", x="LOSClass2", data=df_female, ci=95, join=False);
plt.xlabel("LOS_7DAYS")
plt.ylabel("Number of Hospital Days in Previous year")
plt.show
fig.savefig("prev_days_LOS2.png", dpi=300, bbox_inches= "tight")

#### VALIDITY OF NUM_PRVHOSPITAL_DAYS_1YR IN OUR PREDICTION OF LOS FOR 4 DAYS THRESHOLD

In [378]:
df_female.groupby(['LOSClass', 'Ethnicity'])['NUM_PRVHOSPITAL_DAYS_1YR'].agg(['mean', 'std']).reset_index().to_csv("prev_days_LOS_ethn_stat.csv")

In [379]:
# PREVADMISSIONS _READMIT30DAYS STAT_GENDER Stat
df_female.groupby(['LOSClass', 'GNDR_DESC'])['NUM_PRVHOSPITAL_DAYS_1YR'].agg(['mean', 'std']).reset_index().to_csv("prev_days_LOS_gender_stat.csv")

In [380]:
df_female.groupby(['LOSClass', 'WIMD2019_QUINTILE_AT_INDEX_DATE'])['NUM_PRVHOSPITAL_DAYS_1YR'].agg(['mean', 'std']).reset_index().to_csv("prev_days_LOS_wimd_stat.csv")

In [381]:
df.groupby(['LOSClass'])['NUM_PRVHOSPITAL_DAYS_1YR'].agg(['mean', 'std']).reset_index().to_csv("prev_days_LOS.csv")

In [ ]:
# previous admissions- readmission - ethnicity figure (dodge)
fig = plt.figure()
sns.pointplot(y="NUM_PRVHOSPITAL_DAYS_1YR", x="LOSClass", hue="Ethnicity",
             data=df_female, ci=95, join=False, dodge=True);
plt.xlabel("LOS_4DAYS")
plt.ylabel("Number of Hospital Days in Previous year")
plt.show
fig.savefig("prev_days__LOS_ethn_dodge.png", dpi=300, bbox_inches= "tight")

In [ ]:
# PREVADMISSIONS _READMIT30DAYS WIMD2019_QUINTILE_AT_INDEX_DATE Figure (dodge)
fig = plt.figure()
aa = sns.pointplot(y="NUM_PRVHOSPITAL_DAYS_1YR", x="LOSClass", hue="WIMD2019_QUINTILE_AT_INDEX_DATE",
             data=df_female, ci=95, dodge=True,  join=False);
plt.xlabel("LOS_4DAYS")
plt.ylabel("Number of Hospital Days in Previous year")
sns.move_legend(aa, 'upper left', bbox_to_anchor=(1,1))
plt.legend(title="WIMD 2019 - Quintile")
plt.show
fig.savefig("prev_days__LOS_wimd_dodge.png", dpi=300, bbox_inches= "tight")

In [ ]:
fig = plt.figure()
sns.pointplot(y="NUM_PRVHOSPITAL_DAYS_1YR", x="LOSClass", data=df_female, ci=95, join=False);
plt.xlabel("LOS_4DAYS")
plt.ylabel("Number of Hospital Days in Previous year")
plt.show
fig.savefig("prev_days_LOS.png", dpi=300, bbox_inches= "tight")

#### VALIDITY OF TOTAL COMORBIDITY_COUNT IN OUR PREDICTION OF LOS FOR 4 DAYS THRESHOLD

In [386]:
df_female.groupby(['LOSClass', 'Ethnicity'])['COMORBIDITY_COUNT'].agg(['mean', 'std']).reset_index().to_csv("Comorbid_LOS_ethn_stat.csv")

In [387]:
# PREVADMISSIONS _READMIT30DAYS STAT_GENDER Stat
df_female.groupby(['LOSClass', 'GNDR_DESC'])['COMORBIDITY_COUNT'].agg(['mean', 'std']).reset_index().to_csv("Comorbid_LOS_gender_stat.csv")

In [388]:
df_female.groupby(['LOSClass', 'WIMD2019_QUINTILE_AT_INDEX_DATE'])['COMORBIDITY_COUNT'].agg(['mean', 'std']).reset_index().to_csv("Comorbid_LOS_wimd_stat.csv")

In [389]:
df_female.groupby(['LOSClass'])['COMORBIDITY_COUNT'].agg(['mean', 'std']).reset_index().to_csv("Comorbid_LOS.csv")

In [ ]:
# previous admissions- readmission - ethnicity figure (dodge)
fig = plt.figure()
sns.pointplot(y="COMORBIDITY_COUNT", x="LOSClass", hue="Ethnicity",
             data=df_female, ci=95, join=False, dodge=True);
plt.xlabel("LOS_4DAYS")
plt.ylabel("Number of Comorbidities at Admission")
plt.show
fig.savefig("Comorbid__LOS_ethn_dodge.png", dpi=300, bbox_inches= "tight")

In [ ]:
# PREVADMISSIONS _READMIT30DAYS STAT_GENDER figure normal
fig = plt.figure()
aa = sns.pointplot(y="COMORBIDITY_COUNT", x="LOSClass", hue="GNDR_DESC",
             data=df_female, ci=95, join=False, dodge=True);

plt.xlabel("LOS_4DAYS")
plt.ylabel("Number of Comorbidities at Admission")
sns.move_legend(aa, loc='upper left', bbox_to_anchor=(1,1))
#plt.legend(title="Gender")
plt.show
fig.savefig("Comorbid__LOS_gender.png", dpi=300, bbox_inches= "tight")

In [ ]:
# PREVADMISSIONS _READMIT30DAYS WIMD2019_QUINTILE_AT_INDEX_DATE Figure (dodge)
fig = plt.figure()
aa = sns.pointplot(y="COMORBIDITY_COUNT", x="LOSClass", hue="WIMD2019_QUINTILE_AT_INDEX_DATE",
             data=df_female, ci=95, dodge=True,  join=False);
plt.xlabel("LOS_4DAYS")
plt.ylabel("Number of Comorbidities at Admission")
sns.move_legend(aa, 'upper left', bbox_to_anchor=(1,1))
plt.legend(title="WIMD 2019 - Quintile")
plt.show
fig.savefig("Comorbid__LOS_wimd_dodge.png", dpi=300, bbox_inches= "tight")

In [ ]:
fig = plt.figure()
sns.pointplot(y="COMORBIDITY_COUNT", x="LOSClass", data=df, ci=95, join=False);
plt.xlabel("LOS_4DAYS")
plt.ylabel("Number of Comorbidities at Admission")
plt.show
fig.savefig("Comorbid_LOS.png", dpi=300, bbox_inches= "tight")

#### VALIDITY OF TOTAL COMORBIDITIES WITHIN 24 HOURS OF ADMISSION IN OUR PREDICTION OF LOS FOR 4 DAYS THRESHOLD

In [394]:
df_female.groupby(['LOSClass', 'Ethnicity'])['NUMCOMORBIDITIES_24HRS'].agg(['mean', 'std']).reset_index().to_csv("NumComorbid24hr_LOS_ethn_stat.csv")

In [395]:
# PREVADMISSIONS _READMIT30DAYS STAT_GENDER Stat
df_female.groupby(['LOSClass', 'GNDR_DESC'])['NUMCOMORBIDITIES_24HRS'].agg(['mean', 'std']).reset_index().to_csv("NumComorbid24hr_LOS_gender_stat.csv")

In [396]:
df_female.groupby(['LOSClass', 'WIMD2019_QUINTILE_AT_INDEX_DATE'])['NUMCOMORBIDITIES_24HRS'].agg(['mean', 'std']).reset_index().to_csv("NumComorbid24hr_LOS_wimd_stat.csv")

In [397]:
df_female.groupby(['LOSClass'])['NUMCOMORBIDITIES_24HRS'].agg(['mean', 'std']).reset_index().to_csv("NumComorbid24hr_LOS.csv")

In [ ]:
# previous admissions- readmission - ethnicity figure (dodge)
fig = plt.figure()
sns.pointplot(y="NUMCOMORBIDITIES_24HRS", x="LOSClass", hue="Ethnicity",
             data=df_female, ci=95, join=False, dodge=True);
plt.xlabel("LOS$\geq$4DAYS")
plt.ylabel("NUMCOMORBIDITIES_24HRS")
# plt.title("Number of Conditions Related to Episodes Occuring within 24Hours of Admission")
plt.show
fig.savefig("NumComorbid24hr__LOS_ethn_dodge_FEMALE.png", dpi=300, bbox_inches= "tight")

In [ ]:
# PREVADMISSIONS _READMIT30DAYS STAT_GENDER figure normal
fig = plt.figure()
aa = sns.pointplot(y="NUMCOMORBIDITIES_24HRS", x="LOSClass", hue="GNDR_DESC",
             data=df_female, ci=95, join=False, dodge=True);

plt.xlabel("LOS_4DAYS")
plt.ylabel("Number of Conditions")
plt.title("Number of Conditions Related to Episodes Occuring within 24Hours of Admission")
sns.move_legend(aa, loc='upper left', bbox_to_anchor=(1,1))
plt.legend(title="Gender")
plt.show
fig.savefig("NumComorbid24hr__LOS_gender.png", dpi=300, bbox_inches= "tight")

In [ ]:
# PREVADMISSIONS _READMIT30DAYS WIMD2019_QUINTILE_AT_INDEX_DATE Figure (dodge)
fig = plt.figure()
aa = sns.pointplot(y="NUMCOMORBIDITIES_24HRS", x="LOSClass", hue="WIMD2019_QUINTILE_AT_INDEX_DATE",
             data=df_female, ci=95, dodge=True,  join=False);
plt.xlabel("LOS_4DAYS")
plt.ylabel("Number of Conditions")
plt.title("Number of Conditions Related to Episodes Occuring within 24Hours of Admission")
sns.move_legend(aa, 'upper left', bbox_to_anchor=(1,1))
plt.legend(title="WIMD 2019 - Quintile")
plt.show
fig.savefig("NumComorbid24hr__LOS_wimd_dodge.png", dpi=300, bbox_inches= "tight")

In [ ]:
fig = plt.figure()
sns.pointplot(y="NUMCOMORBIDITIES_24HRS", x="LOSClass", data=df_female, ci=95, join=False);
plt.xlabel("LOS_4DAYS")
plt.ylabel("Number of Conditions")
plt.title("Number of Conditions Related to Episodes Occuring within 24Hours of Admission")
plt.show
fig.savefig("NumComorbid24hr_LOS.png", dpi=300, bbox_inches= "tight")

#### VALIDITY OF NUMBER OF HOSPITAL EPISODES WITHIN 24 HOURS OF ADMISSION IN OUR PREDICTION OF LOS FOR 4 DAYS THRESHOLD

In [55]:
df_female.groupby(['LOSClass', 'Ethnicity'])['NUMEPISODES_24HRS'].agg(['mean', 'std']).reset_index().to_csv("NumEPI24hr_LOS_ethn_stat.csv")

In [56]:
# PREVADMISSIONS _READMIT30DAYS STAT_GENDER Stat
df_female.groupby(['LOSClass', 'GNDR_DESC'])['NUMEPISODES_24HRS'].agg(['mean', 'std']).reset_index().to_csv("NumEPI24hr_LOS_gender_stat.csv")

In [57]:
df_female.groupby(['LOSClass', 'WIMD2019_QUINTILE_AT_INDEX_DATE'])['NUMEPISODES_24HRS'].agg(['mean', 'std']).reset_index().to_csv("NumEPI24hr_LOS_wimd_stat.csv")

In [58]:
df_female.groupby(['LOSClass'])['NUMCOMORBIDITIES_24HRS'].agg(['mean', 'std']).reset_index().to_csv("NumEPI24hr_LOS.csv")

In [ ]:
# previous admissions- readmission - ethnicity figure (dodge)
fig = plt.figure()
sns.pointplot(y="NUMEPISODES_24HRS", x="LOSClass", hue="Ethnicity",
             data=df_female, ci=95, join=False, dodge=True);
plt.xlabel("LOS$\geq$4DAYS")
plt.ylabel("NUMEPISODES_24HRS")
#plt.title("Number of Conditions Related to Episodes Occuring within 24Hours of Admission")
plt.show
fig.savefig("NumEPI24hr__LOS_ethn_dodge_FEMALE.png", dpi=300, bbox_inches= "tight")

In [ ]:
# PREVADMISSIONS _READMIT30DAYS STAT_GENDER figure normal
fig = plt.figure()
aa = sns.pointplot(y="NUMEPISODES_24HRS", x="LOSClass",
             data=df_female, ci=95, join=False, dodge=True);

plt.xlabel("LOS$\geq$4DAYS")
plt.ylabel("NUMEPISODES_24HRS")
# plt.title("Number #of Conditions Related to Episodes Occuring within 24Hours of Admission")
# sns.move_legend(aa, loc='upper left', bbox_to_anchor=(1,1))
# plt.legend(title="Gender")
plt.show
fig.savefig("NumEPI24hr__LOS_gender.png", dpi=300, bbox_inches= "tight")

In [ ]:
fig = plt.figure()
sns.pointplot(y="NUMEPISODES_24HRS", x="LOSClass", data=df_female, ci=95, join=False);
plt.xlabel("LOS$\geq$4DAYS")
plt.ylabel("NUMEPISODES_24HRS")
#plt.title("Number of Conditions Related to Episodes Occuring within 24Hours of Admission")
plt.show
fig.savefig("NumEPI24hr_LOS.png", dpi=300, bbox_inches= "tight")

## Prepare training and test datasets

As  mentioned in the task definition, the target variable is **LOS**, and our sensitive feature for the purposes of fairness assessment is **Ethnicity**.


In [48]:
df_female.columns

Index(['ALF_PE', 'CONDITION', 'COND', 'GNDR_DESC',
       'WIMD2019_QUINTILE_AT_INDEX_DATE',
       'TOWNSEND2011_QUINTILE_AT_INDEX_DATE', 'Ethnicity', 'LOS', 'COM_SEQ',
       'COMORBIDITY_COUNT', 'AUTISM', 'ALCOHOL_HISTORY', 'SMOKING_HISTORY',
       'MEDICATIONS', 'PHYSICAL', 'BMI', 'AGEGRP_AT_ADMIS_DT',
       'NUM_PRVEPISODES_1YR', 'NUM_PRVCOMORBID_1YR', 'NUM_PRVCOMORBID_3YR',
       'NUM_PRVEPISODES_3YR', 'NUMEPISODES_24HRS', 'NUMCOMORBIDITIES_24HRS',
       'NUM_PRVADMISSION_1YR_COND', 'NUM_PRVADMISSION_3YR_COND',
       'NUM_PRVADMISSION_1YR', 'NUM_PRVADMISSION_3YR',
       'NUM_PRVHOSPITAL_DAYS_1YR', 'NUM_PRVHOSPITAL_DAYS_3YR', 'LOSClass',
       'LOSClass2', 'TOTAL_COMORBIDITY'],
      dtype='object')

In [49]:
# df_female.to_csv("ML_ALLDATA_FEMALE.csv")

In [341]:
df_ml = df_female.drop(columns=['LOS', 'LOSClass2', 'TOWNSEND2011_QUINTILE_AT_INDEX_DATE', 'ALF_PE', 'CONDITION', 'COM_SEQ', 'COMORBIDITY_COUNT','NUM_PRVADMISSION_1YR_COND',
                              'NUM_PRVADMISSION_3YR_COND'])

In [342]:
df_ml.columns

Index(['COND', 'GNDR_DESC', 'WIMD2019_QUINTILE_AT_INDEX_DATE', 'Ethnicity',
       'AUTISM', 'ALCOHOL_HISTORY', 'SMOKING_HISTORY', 'MEDICATIONS',
       'PHYSICAL', 'BMI', 'AGEGRP_AT_ADMIS_DT', 'NUM_PRVEPISODES_1YR',
       'NUM_PRVCOMORBID_1YR', 'NUM_PRVCOMORBID_3YR', 'NUM_PRVEPISODES_3YR',
       'NUMEPISODES_24HRS', 'NUMCOMORBIDITIES_24HRS', 'NUM_PRVADMISSION_1YR',
       'NUM_PRVADMISSION_3YR', 'NUM_PRVHOSPITAL_DAYS_1YR',
       'NUM_PRVHOSPITAL_DAYS_3YR', 'LOSClass', 'TOTAL_COMORBIDITY'],
      dtype='object')

In [343]:
df_ml.to_csv("ML_ALLDATA_FEMALE.csv")

In [345]:
target_variable = "LOSClass"
demographic = ["GNDR_DESC", "AGEGRP_AT_ADMIS_DT", "Ethnicity", 'WIMD2019_QUINTILE_AT_INDEX_DATE']
sensitive = ["Ethnicity", "WIMD2019_QUINTILE_AT_INDEX_DATE", "GNDR_DESC", "AGEGRP_AT_ADMIS_DT"]
# sensitive = ["Ethnicity"]
# sensitive = ["AGEGRP_AT_ADMIS_DT"]
# If multiple sensitive features are chosen, the rest of the script considers intersectional groups.

In [346]:
Y, A = df_ml.loc[:, target_variable], df_ml.loc[:, sensitive]

In [ ]:
df_ml.shape

We next drop the features that we don't want to use in our model and expand the categorical features into 0/1 indicators ("dummies").

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb= MultiLabelBinarizer()
dum= mlb.fit_transform(df_ml.COND)

df_ml=df_ml.join(pd.DataFrame(dum.astype(bool), df_ml.index, mlb.classes_))
df_ml

In [350]:
X = pd.get_dummies(df_ml.drop(columns=['LOSClass', 'Ethnicity', 'WIMD2019_QUINTILE_AT_INDEX_DATE', 'GNDR_DESC','COND']))

We split our data into a training and test portion. The test portion will be used to evaluate our performance metric (i.e., balanced accuracy), but also for fairness assessment. The split is half/half for training and test to ensure that we have sufficient sample sizes for fairness assessment.

In [ ]:
# normalise the non-categorical features to be within 0 and 1
from sklearn.preprocessing import MinMaxScaler

for col in X.columns:
  if len(X[col].unique()) > 2:
    X[col] = MinMaxScaler().fit_transform(X[col].values.reshape(-1,1))

X.head()

In [354]:
# X.to_csv("ML_INPUT_FEMALE.csv")

In [355]:
random_seed = 445
np.random.seed(random_seed)

In [356]:
X_train, X_test, Y_train, Y_test, A_train, A_test, df_train, df_test = train_test_split(
    X,
    Y,
    A,
    df_ml,
    test_size=0.50,
    stratify=Y,
    random_state=random_seed
    # random_state=None
)

In [ ]:
Y_train.value_counts()




Our performance metric is **balanced accuracy**, so for the purposes of training (but not evaluation!) we will resample the data set, so that it has the same number of positive and negative examples. This means that we can use estimators that optimize standard accuracy (although some estimators allow the use us importance weights).


Because we are downsampling the number of negative examples, we create a training dataset with a significantly lower number of data points. For more complex machine learning models, this lower number of training data points may affect the model's accuracy.

In [358]:
def resample_dataset(X_train, Y_train, A_train):

  negative_ids = Y_train[Y_train == 0].index
  positive_ids = Y_train[Y_train == 1].index
  balanced_ids = positive_ids.union(np.random.choice(a=negative_ids, size=len(positive_ids)))

  X_train = X_train.loc[balanced_ids, :]
  Y_train = Y_train.loc[balanced_ids]
  A_train = A_train.loc[balanced_ids, :]
  return X_train, Y_train, A_train

In [359]:
X_train_bal, Y_train_bal, A_train_bal = resample_dataset(X_train, Y_train, A_train)

## Save descriptive statistics of training and test data

We next evaluate and save descriptive statistics of the training dataset. These will be provided as part of _model cards_ to document our training.

### Obtain demographic counts of test and train data for target variable 

In [ ]:
X_train_bal.shape

In [ ]:
Y_train_bal.value_counts()

In [ ]:
A_train_bal['AGEGRP_AT_ADMIS_DT'].value_counts()

In [ ]:
Y_train_balc.loc[indices, :]['LOSClass'].value_counts()

In [ ]:
Y_test.shape

In [ ]:
Y_test.value_counts()

In [ ]:
A_test['AGEGRP_AT_ADMIS_DT'].value_counts()

In [ ]:
Y_testc.loc[indices, :]['LOSClass'].value_counts()

In [ ]:
ax = sns.countplot(x="Ethnicity", data=A_train_bal)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
ax.set_yscale('log')
plt.title("Sensitive Attributes for Training Dataset")
plt.xlabel("Ethnicity")
plt.ylabel("Count(Log Scale)")
plt.savefig("train_ethn.png", dpi=300, bbox_inches= "tight")
# sensitive_train = figure_to_base64str(plt)

In [ ]:
ax = sns.countplot(x="WIMD2019_QUINTILE_AT_INDEX_DATE", data=A_train_bal)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
plt.title("Sensitive Attributes for Training Dataset")
plt.xlabel("WIMD 2019 - Quintile")
plt.ylabel("Count")
plt.savefig("train_wimd.png", dpi=300, bbox_inches= "tight")
# sensitive_train = figure_to_base64str(plt)

In [38]:
A_train_bal['Ethnicity'].value_counts().to_csv('train_ethn.csv')

In [ ]:
A_train_bal['WIMD2019_QUINTILE_AT_INDEX_DATE'].value_counts().to_csv('train_wimd.csv')

In [451]:
A_train_bal['GNDR_DESC'].value_counts().to_csv('train_gender.csv')

In [452]:
Y_train_bal.value_counts().to_csv("train_LOS_count.csv")

In [ ]:
plot = sns.countplot(x=Y_train_bal)
plot.set_xlabel("LOS_4DAYS")
plt.title("Target Label Histogram for Training Dataset")
plt.ylabel("Count")
plt.savefig("train_LOS_count.png", dpi=300, bbox_inches= "tight")
# outcome_train = figure_to_base64str(plt)

In [ ]:
sns.countplot(x="Ethnicity", data=A_test)
plt.title("Sensitive Attributes for Testing Dataset")
plt.ylabel("Count")
plt.savefig('test_ethin_wo_log.png', dpi=300, bbox_inches= "tight")
# sensitive_test = figure_to_base64str(plt)

In [ ]:
ax = sns.countplot(x="Ethnicity", data=A_test)
plt.title("Sensitive Attributes for Testing Dataset")
ax.set_yscale('log')
plt.ylabel("Count(Log Scale)")
plt.savefig('test_ethin_log.png', dpi=300, bbox_inches= "tight")
# sensitive_test = figure_to_base64str(plt)

In [86]:
A_test['Ethnicity'].value_counts().to_csv("test_ethin.csv")

In [ ]:
sns.countplot(x="WIMD2019_QUINTILE_AT_INDEX_DATE", data=A_test)
plt.title("Sensitive Attributes for Testing Dataset")
plt.xlabel("WIMD 2019 - Quintile")
plt.ylabel("Count")
plt.savefig('test_widm_wo_log.png', dpi=300, bbox_inches= "tight")
# sensitive_test = figure_to_base64str(plt)

In [ ]:
ax = sns.countplot(x="WIMD2019_QUINTILE_AT_INDEX_DATE", data=A_test)
plt.title("Sensitive Attributes for Testing Dataset")
ax.set_yscale('log')
plt.ylabel("Count (Log Scale) ")
plt.xlabel("WIMD 2019 - Quintile")
plt.savefig('test_widm_log.png', dpi=300, bbox_inches= "tight")
# sensitive_test = figure_to_base64str(plt)

In [89]:
A_test['WIMD2019_QUINTILE_AT_INDEX_DATE'].value_counts().to_csv("test_widm.csv")

In [ ]:
sns.countplot(x=Y_test)
plt.title("Target Label Histogram for Test Dataset")
plt.xlabel('LOS_4DAYS')
plt.ylabel("Count")
plt.savefig("test_readm.png")
# outcome_test = figure_to_base64str(plt)

In [464]:
Y_test.value_counts().to_csv("test_LOS_count.csv")

## Train the model

We train a logistic regression model and save its predictions on test data for analysis.

In [197]:


unmitigated_pipeline = Pipeline(steps=[
    ("preprocessing", StandardScaler()),
    ("logistic_regression", LogisticRegression(max_iter=1000))
])

# from sklearn import svm
# unmitigated_pipeline=XGBClassifier(eval_metric='mlogloss')

In [198]:
unmitigated_pipeline.fit(X_train_bal, Y_train_bal)
# unmitigated_pipeline.fit(X_train, Y_train)

Pipeline(steps=[('preprocessing', StandardScaler()),
                ('logistic_regression', LogisticRegression(max_iter=1000))])

In [199]:
Y_pred_proba = unmitigated_pipeline.predict_proba(X_test)[:,1]
Y_pred = unmitigated_pipeline.predict(X_test)

Check model performance on test data.

In [ ]:
# Plot ROC curve of probabilistic predictions
display = RocCurveDisplay.from_estimator(unmitigated_pipeline, X_test, Y_test)
display.plot()
plt.savefig("roccurve.png", dpi=300, bbox_inches= "tight")

In [ ]:
# Show balanced accuracy rate of the 0/1 predictions
balanced_accuracy_score(Y_test, Y_pred)

In [ ]:
false_negative_rate(Y_test, Y_pred)

In [ ]:
false_positive_rate(Y_test, Y_pred)

In [ ]:
selection_rate(Y_test, Y_pred)

In [205]:
AUC_LINEAR=roc_auc_score(Y_test, Y_pred_proba)

In [ ]:
AUC_LINEAR

In [206]:
fpr,tpr, threshold= roc_curve(Y_test,Y_pred_proba)

In [ ]:
from numpy import sqrt
gmeans=sqrt(tpr*(1-fpr))
ix=np.argmax(gmeans)
threshold[ix]

In [208]:
fpr_OPT_LINEAR=fpr[ix]

In [209]:
tpr_OPT_LINEAR=tpr[ix]

## Compare to other models

### RUN NEURAL NETWORK ON DATA

In [210]:
# from pyvis.network import Network
# import networkx as nx
from numpy import save
from numpy import load
import random
from numpy import newaxis
from numpy import array
from numpy import array
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
torch.manual_seed(0)
from torch import nn
torch.pi = torch.acos(torch.zeros(1)).item() * 2 # which is 3.1415927410125732  
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import MinMaxScaler
import math
import array
import copy


from pytorch_model_summary import summary

print ('import completed')



import completed


In [211]:
N= X.shape[1] #Get input size of NN

# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))


# Defining the Network model for 16 nodes
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        """this is the neural network 
        it can take a shape of N columns by any rows
        the output layer is 2
        """
        #NN MODEL
        self.m0 = nn.Sequential(
            nn.Linear(N,N),
            nn.ReLU(),
            # nn.Linear(2*N,2*N),
            # nn.ReLU(),
            nn.Linear(N,2),
            # nn.Softmax(dim=1),
        )
        
    #FORWARD PASS FOR NN
    def forward(self, x):
        #x is input data that you want to pass into the neural network
        # self.network_output is network output
        self.network_output0 = self.m0(x)
        return self.network_output0
    
model = NeuralNetwork().to(device)

Using cpu device


In [212]:
loss_fn =  nn.CrossEntropyLoss()
# loss_fn =  nn.BCELoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)
optimizer = torch.optim.SGD(model.parameters(), lr=0.0008, momentum=0.9, weight_decay=0.01)
scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

In [ ]:
Y_train_NN = pd.get_dummies(Y_train_bal, dtype=float).reset_index(drop=True)
# Y_train_NN = pd.get_dummies(Y_train, dtype=float).reset_index(drop=True)

Y_test_NN = pd.get_dummies(Y_test, dtype=float).reset_index(drop=True)

# convert input DataFrame (X) and output Dataframe (y) into PyTorch tensors
X_train_NN = torch.tensor(X_train_bal.values.astype('float'), dtype=torch.float32,requires_grad=True)
# X_train_NN = torch.tensor(X_train.values.astype('float'), dtype=torch.float32,requires_grad=True)

Y_train_NN = torch.tensor(Y_train_NN.values, dtype=torch.float32)


X_test_NN = torch.tensor(X_test.values.astype('float'), dtype=torch.float32)
Y_test_NN = torch.tensor(Y_test_NN.values, dtype=torch.float32)


n_epochs = 200
batch_size = 100
batches_per_epoch = len(X_train_NN) // batch_size


best_bal_acc = - np.inf   # init to negative infinity
best_weights = None
train_loss_hist = []
train_acc_hist = []
train_bal_acc_hist= []
test_loss_hist = []
test_acc_hist = []
test_bal_acc_hist= []

for epoch in range(n_epochs):
    epoch_loss = []
    epoch_acc = []
    epoch_bal_acc= []
    # set model in training mode and run through each batch
    model.train()
    for i in range(batches_per_epoch):
        # take a batch
        start = i * batch_size
        X_batch = X_train_NN[start:start+batch_size]
        y_batch = Y_train_NN[start:start+batch_size]
        # forward pass
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        # update weights
        optimizer.step()
        # compute and store metrics
        acc = (torch.argmax(y_pred, 1) == torch.argmax(y_batch, 1)).float().mean()
        bal_acc=balanced_accuracy_score(torch.argmax(y_batch,1), torch.argmax(y_pred, 1))
        epoch_loss.append(float(loss))
        epoch_acc.append(float(acc))
        epoch_bal_acc.append(float(bal_acc))


    train_loss_hist.append(np.mean(epoch_loss))
    train_acc_hist.append(np.mean(epoch_acc))
    train_bal_acc_hist.append(np.mean(epoch_bal_acc))
    # set model in evaluation mode and run through the test set
    model.eval()
    y_pred = model(X_test_NN)
    ce = loss_fn(y_pred, Y_test_NN)
    acc = (torch.argmax(y_pred, 1) == torch.argmax(Y_test_NN, 1)).float().mean()
    bal_acc=balanced_accuracy_score(torch.argmax(Y_test_NN,1), torch.argmax(y_pred, 1))
    ce = float(ce)
    acc = float(acc)
    bal_acc= float(bal_acc)

    test_loss_hist.append(ce)
    test_acc_hist.append(acc)
    test_bal_acc_hist.append(bal_acc)
    if bal_acc > best_bal_acc:
        best_bal_acc = bal_acc
        best_weights = copy.deepcopy(model.state_dict())
    print(f"Epoch {epoch} validation: Cross-entropy={ce}, Bal_Accuracy={bal_acc}")

model.load_state_dict(best_weights)

RUN OTHER CLASSIFICATION MODELS

The charts above are based on test data, so without any uncertainty quantification (such as error bars or confidence intervals), we cannot reliably compare these data statistics. Next optional section shows how to augment MetricFrame with the report of error bars.

## Adding error bars [OPTIONAL SECTION]

In this section, we define new custom metrics that quantify errors in our estimates of selection rate, false negative rate and balanced accuracy, and then review our metrics again.

In [176]:
# All of our error bar calculations are based on normal approximation to
# the binomial variables.

def error_bar_normal(n_successes, n_trials, z=1.96):
  """
  Computes the error bars for the parameter p of a binomial variable
  using normal approximation. The default value z corresponds to the 95%
  confidence interval.
  """
  point_est = n_successes / n_trials
  error_bar = z*np.sqrt(point_est*(1-point_est))/np.sqrt(n_trials)
  return error_bar

def fpr_error(Y_true, Y_pred):
  """
  Compute the 95%-error bar for the false positive rate
  """
  tn, fp, fn, tp = confusion_matrix(Y_true, Y_pred).ravel()
  return error_bar_normal(fp, tn+fp)

def fnr_error(Y_true, Y_pred):
  """
  Compute the 95%-error bar for the false negative rate
  """
  tn, fp, fn, tp = confusion_matrix(Y_true, Y_pred).ravel()
  return error_bar_normal(fn, fn+tp)

def selection_rate_error(Y_true, Y_pred):
  """
  Compute the 95%-error bar for the selection rate
  """
  tn, fp, fn, tp = confusion_matrix(Y_true, Y_pred).ravel()
  return error_bar_normal(tp+fp, tn+fp+fn+tp)

def balanced_accuracy_error(Y_true, Y_pred):
  """
  Compute the 95%-error bar for the balanced accuracy
  """
  fnr_err, fpr_err = fnr_error(Y_true, Y_pred), fpr_error(Y_true, Y_pred)
  return np.sqrt(fnr_err**2 + fpr_err**2)/2

We next create a metric frame that includes the sample sizes and error bar sizes in addition to the metrics that we have used previously.

In [177]:
metrics_with_err_bars = {
    "count": count,
    "selection_rate": selection_rate,
    "selection_err_bar": selection_rate_error,
    "false_negative_rate": false_negative_rate,
    "fnr_err_bar": fnr_error,
    "balanced_accuracy": balanced_accuracy_score,
    "bal_acc_err_bar": balanced_accuracy_error
}

# sometimes we will only want to display metrics without error bars
metrics_to_display = [
    "count",
    "selection_rate",
    "false_negative_rate",
    "balanced_accuracy"
]

# sometimes we will only want to show the difference values of the metrics other than count
differences_to_display = [
    "selection_rate",
    "false_negative_rate",
    "balanced_accuracy"
]

In [178]:
metricframe_unmitigated_w_err = MetricFrame(
    metrics=metrics_with_err_bars,
    y_true=Y_test,
    y_pred=Y_pred,
    sensitive_features=A_test
)

In [ ]:
unmitigated_groups = metricframe_unmitigated_w_err.by_group
unmitigated_groups # show both the metrics as well as the error bars

We see that for smaller sample sizes we have larger error bars. The problem is further exacerbated for false negative rate, which is estimated only over *positive examples* and so its sample sizes is further reduced due to label imbalance.

We next visualize the metrics with the corresponding error bars using a custom plotting function.

In [180]:
def plot_group_metrics_with_error_bars(metricframe, metric, error_name):
  """
  Plots the disaggregated `metric` for each group with an associated
  error bar. Both metric and the erro bar are provided as columns in the 
  provided metricframe.
  """
  grouped_metrics = metricframe.by_group
  point_estimates = grouped_metrics[metric]
  error_bars = grouped_metrics[error_name]
  lower_bounds = point_estimates - error_bars
  upper_bounds = point_estimates + error_bars

  x_axis_names = [str(name) for name in error_bars.index.to_flat_index().tolist()]
  plt.vlines(x_axis_names, lower_bounds, upper_bounds, linestyles="dashed", alpha=0.45)
  plt.scatter(x_axis_names, point_estimates, s=25)
  plt.xticks(rotation=0)
  y_start, y_end = np.round(min(lower_bounds), decimals=2), np.round(max(upper_bounds), decimals=2)
  plt.yticks(np.arange(y_start, y_end, 0.05))
  plt.ylabel(metric)

In [ ]:
plot_group_metrics_with_error_bars(metricframe_unmitigated_w_err, "selection_rate", "selection_err_bar")

In [ ]:
plot_group_metrics_with_error_bars(metricframe_unmitigated_w_err, "false_negative_rate", "fnr_err_bar")

In [ ]:
plot_group_metrics_with_error_bars(metricframe_unmitigated_w_err, "balanced_accuracy", "bal_acc_err_bar")

As we see above, even accounting for the larger uncertainty in estimating the false negative rate for *Unknown*, this group is experiencing substantially larger false negative rate than other groups and thus experiences the harm of allocation.



---


# **Mitigating fairness-related harms in ML models**

We have found that the logistic regression predictor leads to a large difference in false negative rates between the groups. We next look at **algorithmic mitigation strategies** of this fairness issue (and similar ones).

*Note that while we currently focus on the training stage of the AI lifecycle mitigation should not be limited to this stage. In fact, we have already discussed mitigation strategies that are applicable at the task definition stage (e.g., checking for construct validity) and data collection stage (e.g., collecting more data).*

Within the model training stage, mitigation may occur at different steps relative to model training:

* **Preprocessing**: A mitigation algorithm is applied to transform the input data to the training algorithm; for example, some strategies seek to remove and dependence between the input features and sensitive features.

* **At training time**: The model is trained by an (optimization) algorithm that seeks to satisfy fairness constraints.

* **Postprocessing**: The output of a trained model is transformed to mitigate fairness issues; for example, the predicted probability of readmission is thresholded according to a group-specific threshold.

We will now dive into two algorithms: a postprocessing approach and a reductions approach (which is a training-time algorithm). Both of them are in fact **meta-algorithms** in the sense that they act as wrappers around *any* standard (fairness-unaware) machine learning algorithms. This makes them quite versatile in practice.


## Postprocessing with `ThresholdOptimizer`

**Postprocessing** techniques are a class of unfairness-mitigation algorithms that take an already trained model and a dataset as an input and seek to fit a transformation function to model's outputs to satisfy some (group) fairness constraint(s). They might be the only feasible unfairness mitigation approach when developers cannot influence training of the model, due to practical reasons or due to security or privacy.


Here we use the `ThresholdOptimizer` algorithm from Fairlearn, which follows the approach of [Hardt, Price, and Srebro (2016)](https://arxiv.org/abs/1610.02413).

`ThresholdOptimizer` takes in an exisiting (possibly pre-fit) machine learning model whose predictions act as a scoring function and identifies a separate thrceshold for each group in order to optimize some specified objective metric (such as **balanced accuracy**) subject to specified fairness constraints (such as **false negative rate parity**). Thus, the resulting classifier is just a suitably thresholded version of the underlying machinelearning model.

The constraint **false negative rate parity** requires that all the groups have equal values of false negative rate.



To instatiate our `ThresholdOptimizer`, we pass in:

*   An existing `estimator` that we wish to threshold. 
*   The fairness `constraints` we want to satisfy.
*   The `objective` metric we want to maximize.



In [260]:
# Now we instantite ThresholdOptimizer with the logistic regression estimator
postprocess_est = ThresholdOptimizer(
    estimator=unmitigated_pipeline,
    constraints="false_negative_rate_parity",
    objective="balanced_accuracy_score",
    prefit=True,
    predict_method='predict_proba'
)

In order to use the `ThresholdOptimizer`, we need access to the sensitive features **both during training time and once it's deployed**.

In [261]:
postprocess_est.fit(X_train_bal, Y_train_bal, sensitive_features=A_train_bal)

ThresholdOptimizer(constraints='false_negative_rate_parity',
                   estimator=Pipeline(steps=[('preprocessing',
                                              StandardScaler()),
                                             ('Random_Forest',
                                              RandomForestClassifier())]),
                   objective='balanced_accuracy_score',
                   predict_method='predict_proba', prefit=True)

In [262]:
# Record and evaluate the output of the trained ThresholdOptimizer on test data

Y_pred_postprocess = postprocess_est.predict(X_test, sensitive_features=A_test)
metricframe_postprocess = MetricFrame(
    metrics=metrics_dict,
    y_true=Y_test,
    y_pred=Y_pred_postprocess,
    sensitive_features=A_test
)

We can now inspect how the metric values differ between the postprocessed model and the unmitigated model:

In [ ]:
pd.concat([metricframe_unmitigated.by_group,
           metricframe_postprocess.by_group],
           keys=['Unmitigated', 'ThresholdOptimizer'],
           axis=1)

In [264]:
pd.concat([metricframe_unmitigated.by_group,
           metricframe_postprocess.by_group],
           keys=['Unmitigated', 'ThresholdOptimizer'],
           axis=1).to_csv('thresholdopt_male.csv')

We next zoom in on differences between the largest and the smallest metric values:

In [ ]:
pd.concat([metricframe_unmitigated.difference(),
           metricframe_postprocess.difference()],
          keys=['Unmitigated: difference', 'ThresholdOptimizer: difference'],
          axis=1).T

Finally, we save the disagregated statistics:

In [ ]:
metricframe_postprocess.by_group.plot.bar(subplots=True, layout=[1,4], figsize=(12, 4), legend=False, rot=-45, position=1.5)
# postprocess_performance = figure_to_base64str(plt)

Next optional section shows that `ThresholdOptimizer` more closely satisfies constraints on the training data than on the test data.

### Postprocessing: Correctness check [OPTIONAL SECTION]

We can verify that `ThresholdOptimizer` achieves false negative rate parity on the training dataset, meaning that the values of the false negative rate parity with respect to all groups are close on the training data.

In [ ]:
# Record and evaluate the output of the ThresholdOptimizer on the training data

Y_pred_postprocess_training = postprocess_est.predict(X_train_bal, sensitive_features=A_train_bal)
metricframe_postprocess_training = MetricFrame(
    metrics=metrics_dict,
    y_true=Y_train_bal,
    y_pred=Y_pred_postprocess_training,
    sensitive_features=A_train_bal
)
metricframe_postprocess_training.by_group

In [ ]:
# Evaluate the difference between the largest and smallest value of each metric
metricframe_postprocess_training.difference()

The value of `false_negative_rate_difference` on the training data is smaller than on the test data.

<a name="exercise-threshold"></a>
### Exercise: ThresholdOptimizer

In this exercise, we will create a `ThresholdOptimizer` by constraining the *true positive rate* (also known as the *recall score*). For any model, the *true positive rate* + *false negative rate* = 1. 

By trying to achieve the *true positive rate parity*, we should produce a `ThresholdOptimizer` with the same performance as our original `ThresholdOptimizer`.



#### 1.) Create a new ThresholdOptimizer with the constraint `false_positive_rate_parity` and objective `balanced_accuracy_score`.

In [192]:
thresopt_exercise = ThresholdOptimizer(
    estimator=unmitigated_pipeline,
    constraints="false_positive_rate_parity",
    objective="balanced_accuracy_score",
    prefit=True,
    predict_method='predict_proba'
)

In [193]:
thresopt_exercise.fit(X_train_bal, Y_train_bal, sensitive_features=A_train_bal)
threshopt_pred = thresopt_exercise.predict(X_test, sensitive_features=A_test)

#### 2.) Create a new `MetricFrame` object to process the results of this classifier.

In [194]:
thresop_metricframe = MetricFrame(
    metrics=metrics_dict,
    y_true=Y_test,
    y_pred=threshopt_pred,
    sensitive_features=A_test
)

#### 3.) Compare the performance of the two `ThresholdOptimizers`.

In [ ]:
# Visualize the performance of the new ThresholdOptimizer
thresop_metricframe.by_group

In [ ]:
# Compare the performance to the original ThresholdOptimizer
metricframe_postprocess.by_group

In [ ]:
pd.concat([metricframe_unmitigated.difference(),
           metricframe_postprocess.difference(),
          thresop_metricframe.difference()],
          keys=['Unmitigated: difference', 'ThresholdOptimizer: difference', 'ThresholdOptimizer_fprParity: Difference'],
          axis=1).T

Similar to many unfairness mitigation approaches, `ThresholdOptimizer` produces randomized classifiers. Next optional section presents a heuristic strategy for converting a randomized `ThresholdOptimizer` into a deterministic one. In our scenario, this heursitic is quite effective and the resulting deterministic classifier has similar performance as the original `ThresholdOptimizer`.

### Deployment considerations: Randomized predictions [OPTIONAL SECTION]

When we were describing `ThresholdOptimizer` we said that it picks a separate threshold for each group. However, that is not quite correct. In fact,`ThresholdOptimizer`, for each group, picks two thresholds that are close to each other (say `threshold0` and `threshold1`) and then, at deployment time, randomizes between the two: choosing `threshold0` with some probability `p0` and `threshold1` with the remaining probability `p1=1-p0` (the specific probabilities are determined during training; for certain kinds of constraints, three thresholds are considered.)

This means that the predictions are randomized. To achieve reproducible randomization, it is possible to provide an argument `random_state` to the `predict` method. However, in some settings, even such reproducible randomization is not acceptable and can be in fact viewed as a fairness issue, because of its arbitrariness.

One derandomization heuristic is to replace the two thresholds by their weighted average, i.e., `threshold = p0*threshold0 + p1*threshold1`. That corresponds to the assumption that the values of the scores between the two thresholds are approximately uniformly distributed. Using this heuristic, we derandomize `ThresholdOptimizer`.



#### Derandomization on `ThresholdOptimizer` model

The randomized model of the `ThresholdOptimizer` is stored as the field
`interpolated_thresholder_` in the fitted ThresholdOptimizer, which is itself a
valid estimator of type `InterpolatedThresholder`:

In [208]:
interpolated = postprocess_est.interpolated_thresholder_
interpolated

InterpolatedThresholder(estimator=Pipeline(steps=[('preprocessing',
                                                   StandardScaler()),
                                                  ('Random_Forest',
                                                   RandomForestClassifier())]),
                        interpolation_dict={'Asian': {'operation0': [>0.4515],
                                                      'operation1': [>0.775],
                                                      'p0': 0.9667857142857142,
                                                      'p1': 0.03321428571428575},
                                            'Black': {'operation0': [>0.528345238095238],
                                                      'operation1': [>inf],
                                                      'p0': 0.995,
                                                      'p1': 0.005000000000000...
                                                      'operation1': [>0.7225],
                                                      'p0': 0.97,
                                                      'p1': 0.030000000000000027},
                                            'Unknown': {'operation0': [>0.46058333333333334],
                                                        'operation1': [>0.5524556277056276],
                                                        'p0': 0.575,
                                                        'p1': 0.42500000000000004},
                                            'White': {'operation0': [>0.45258333333333334],
                                                      'operation1': [>0.5379675324675324],
                                                      'p0': 0.31318181818181806,
                                                      'p1': 0.686818181818182}},
                        predict_method='predict_proba', prefit=True)

The `interpolation_dict` is a dictionary which assign to each sensitive feature value two thresholds and two respective probabilities. Using our derandomization strategy, we can create a dictionary that represents a deterministic rule:


In [209]:
def create_deterministic(interpolate_dict):
  """
  Creates a deterministic interpolation_dictionary from a randomized
  interpolation_dictionary. The determinstic thresholds are created by taking
  the weighted combinations of the two randomized thresholds for each sensitive
  group.
  """
  deterministic_dict = {}
  for (race, operations) in interpolate_dict.items():
    op0, op1 = operations["operation0"]._threshold, operations["operation1"]._threshold
    p0, p1 = operations["p0"], operations["p1"]
    deterministic_dict[race] = Bunch(
      p0=0.0,
      p1=1.0,
      operation0=ThresholdOperation(operator=">",threshold=(p0*op0 + p1*op1)),
      operation1=ThresholdOperation(operator=">",threshold=(p0*op0 + p1*op1))
    )
  return deterministic_dict

In [ ]:
deterministic_dict = create_deterministic(interpolated.interpolation_dict)
deterministic_dict

Now, we can create an `InterpolatedThresholder` that uses the same pre-fit estimator, but with derandomized thresholds.

In [211]:
deterministic_thresholder = InterpolatedThresholder(estimator=interpolated.estimator,
                                                 interpolation_dict=deterministic_dict,
                                                 prefit=True,
                                                 predict_method='predict_proba')

In [212]:
deterministic_thresholder.fit(X_train_bal, Y_train_bal, sensitive_features=A_train_bal)

InterpolatedThresholder(estimator=Pipeline(steps=[('preprocessing',
                                                   StandardScaler()),
                                                  ('Random_Forest',
                                                   RandomForestClassifier())]),
                        interpolation_dict={'Asian': {'operation0': [>0.46224482142857143],
                                                      'operation1': [>0.46224482142857143],
                                                      'p0': 0.0, 'p1': 1.0},
                                            'Black': {'operation0': [>inf],
                                                      'operation1': [>inf],
                                                      'p0': 0.0, 'p1': 1.0},
                                            'Other': {'operation0': [>0.45089999999999997],
                                                      'operation1': [>0.45089999999999997],
                                                      'p0': 0.0, 'p1': 1.0},
                                            'Unknown': {'operation0': [>0.4996290584415584],
                                                        'operation1': [>0.4996290584415584],
                                                        'p0': 0.0, 'p1': 1.0},
                                            'White': {'operation0': [>0.5112267537386855],
                                                      'operation1': [>0.5112267537386855],
                                                      'p0': 0.0, 'p1': 1.0}},
                        predict_method='predict_proba', prefit=True)

In [213]:
y_pred_postprocess_deterministic = deterministic_thresholder.predict(X_test, sensitive_features=A_test)

In [214]:
mf_deterministic = MetricFrame(
    metrics=metrics_dict,
    y_true=Y_test,
    y_pred=y_pred_postprocess_deterministic,
    sensitive_features=A_test
)

Now compare the two models in terms of their disaggregated metrics:

In [ ]:
mf_deterministic.by_group

In [ ]:
metricframe_postprocess.by_group

In [ ]:
pd.concat([metricframe_unmitigated.difference(),
           metricframe_postprocess.difference(),
          thresop_metricframe.difference(),
          mf_deterministic.difference()],
          keys=['Unmitigated: difference', 'ThresholdOptimizer: difference', 'ThresholdOptimizer_fprParity: Difference','DererministicThresholdOptimizer: Difference'],
          axis=1).T

#### Derandomization on the threshold oprimizer model: `thresopt_exercise`  which has the constraint of False_positive_rate parity

The randomized model of the `ThresholdOptimizer` is stored as the field
`interpolated_thresholder_` in the fitted ThresholdOptimizer, which is itself a
valid estimator of type `InterpolatedThresholder`:

In [219]:
interpolated2 = thresopt_exercise.interpolated_thresholder_
interpolated2

InterpolatedThresholder(estimator=Pipeline(steps=[('preprocessing',
                                                   StandardScaler()),
                                                  ('Random_Forest',
                                                   RandomForestClassifier())]),
                        interpolation_dict={'Asian': {'operation0': [>0.4515],
                                                      'operation1': [>0.125],
                                                      'p0': 0.9908378378378379,
                                                      'p1': 0.009162162162162146},
                                            'Black': {'operation0': [>0.528345238095238],
                                                      'operation1': [>-inf],
                                                      'p0': 0.997,
                                                      'p1': 0.0030000000000...
                                                      'operation1': [>-inf],
                                                      'p0': 0.997,
                                                      'p1': 0.0030000000000000027},
                                            'Unknown': {'operation0': [>0.56275],
                                                        'operation1': [>0.5524556277056276],
                                                        'p0': 0.3089999999999999,
                                                        'p1': 0.6910000000000001},
                                            'White': {'operation0': [>0.5379675324675324],
                                                      'operation1': [>0.45258333333333334],
                                                      'p0': 0.1791818181818181,
                                                      'p1': 0.8208181818181819}},
                        predict_method='predict_proba', prefit=True)

The `interpolation_dict` is a dictionary which assign to each sensitive feature value two thresholds and two respective probabilities. Using our derandomization strategy, we can create a dictionary that represents a deterministic rule:


In [ ]:
deterministic_dict2 = create_deterministic(interpolated2.interpolation_dict)
deterministic_dict2

Now, we can create an `InterpolatedThresholder` that uses the same pre-fit estimator, but with derandomized thresholds.

In [221]:
deterministic_thresholder2 = InterpolatedThresholder(estimator=interpolated2.estimator,
                                                 interpolation_dict=deterministic_dict2,
                                                 prefit=True,
                                                 predict_method='predict_proba')

In [222]:
deterministic_thresholder2.fit(X_train_bal, Y_train_bal, sensitive_features=A_train_bal)

InterpolatedThresholder(estimator=Pipeline(steps=[('preprocessing',
                                                   StandardScaler()),
                                                  ('Random_Forest',
                                                   RandomForestClassifier())]),
                        interpolation_dict={'Asian': {'operation0': [>0.4485085540540541],
                                                      'operation1': [>0.4485085540540541],
                                                      'p0': 0.0, 'p1': 1.0},
                                            'Black': {'operation0': [>-inf],
                                                      'operation1': [>-inf],
                                                      'p0': 0.0, 'p1': 1.0},
                                            'Other': {'operation0': [>-inf],
                                                      'operation1': [>-inf],
                                                      'p0': 0.0, 'p1': 1.0},
                                            'Unknown': {'operation0': [>0.5556365887445887],
                                                        'operation1': [>0.5556365887445887],
                                                        'p0': 0.0, 'p1': 1.0},
                                            'White': {'operation0': [>0.4678826293781975],
                                                      'operation1': [>0.4678826293781975],
                                                      'p0': 0.0, 'p1': 1.0}},
                        predict_method='predict_proba', prefit=True)

In [223]:
y_pred_postprocess_deterministic2 = deterministic_thresholder2.predict(X_test, sensitive_features=A_test)

In [224]:
mf_deterministic2 = MetricFrame(
    metrics=metrics_dict,
    y_true=Y_test,
    y_pred=y_pred_postprocess_deterministic2,
    sensitive_features=A_test
)

Now compare the two models in terms of their disaggregated metrics:

In [ ]:
mf_deterministic2.by_group

In [ ]:
mf_deterministic.by_group

In [ ]:
metricframe_postprocess.by_group

In [ ]:
pd.concat([metricframe_unmitigated.difference(),
           metricframe_postprocess.difference(),
           mf_deterministic.difference(),
          thresop_metricframe.difference(),
          mf_deterministic2.difference()],
          keys=['Unmitigated: difference', 'ThresholdOptimizer: difference', 'DererministicThresholdOptimizer: Difference', 'ThresholdOptimizer_fprParity: Difference',
               'DeterministicThresholdOptimizer_fprParity: Difference'],
          axis=1).T

## Reductions approach with `ExponentiatedGradient`

With the `ThresholdOptimizer`, we took a fairness-unaware model and transformed the model's decision boundary to satisfy our fairness constraints. One limitation of `ThresholdOptimizer` is that it needs access to the sensitive features at deployment time.

In this section, we will show how to use the _reductions_ approach of [Agarwal et. al (2018)](https://arxiv.org/abs/1803.02453) to obtain a model that satisfies the fairness constraints, but does not need access to sensitive features at deployment time.

Terminology "reductions" refers to another kind of a wrapper approach, which instead of wrapping an already trained model, wraps any standard classification or regression algorithm, such as 
`LogisticRegression`. In other words, an input to a reduction algorithm is an object that supports training on any provided (weighted) dataset. In addition, a reduction algorithm receives a data set that includes sensitive features. The goal, like with post-processing, is to optimize a performance metric (such as classification accuracy) subject to fairness constraints (such as an upper bound on differences between false negative rates).

The main reduction algorithm algorithm in Fairlearn is `ExponentiatedGradient`. It creates a sequence of reweighted datasets and retrains the wrapped model on each of them. The 
retraining process is guaranteed to find a model that satisfies the fairness constraints while optimizing the performance metric.

The model returned by `ExponentiatedGradient` consists of several inner models, returned by the wrapped estimator. At deployment time, `ExponentiatedGradient` randomizes among these models according to a specific probability weights.

To instantiate an `ExponentiatedGradient` model, we pass in two parameters:

*   A base `estimator` (an object that supports training)
*   Fairness `constraints` (an object of type `Moment`).

The constraints supported by `ExponentiatedGradient` are more general than those supported by `ThresholdOptimizer`. For example, rather than requiring that false negative rates be equal, it is possible to specify the maxium allowed difference or ratio between the largest and the smallest value.


In [266]:
expgrad_est = ExponentiatedGradient(
    estimator=LogisticRegression(max_iter=1000, random_state=random_seed),
    # constraints=TruePositiveRateParity(difference_bound=0.02)
    constraints=FalsePositiveRateParity(difference_bound=0.02)
)

In [267]:
expgrad_est = ExponentiatedGradient(
    estimator=RandomForestClassifier(),
    constraints=TruePositiveRateParity(difference_bound=0.02)
)

The constraints above are expressed for the true positive parity, they require that the difference between the largest and the smallest true positive rate (TPR) across all groups be at most 0.02. Since false negative rate (FNR) is equal to 1-TPR, this is equivalent to requiring that the difference between the largest and smallest FNR be at most 0.02.

In [268]:
# Fit the exponentiated gradient model
expgrad_est.fit(X_train_bal, Y_train_bal, sensitive_features=A_train_bal)

ExponentiatedGradient(constraints=<fairlearn.reductions._moments.utility_parity.TruePositiveRateParity object at 0x7f35b8f8f160>,
                      estimator=RandomForestClassifier(),
                      nu=0.00036129117558525426)

Similarly to `ThresholdOptimizer` the predictions of `ExponentiatedGradient` models are randomized. If we want to assure reproducible results, we can pass  `random_state` to the `predict` function. 

In [ ]:
# Record and evaluate predictions on test data

Y_pred_reductions = expgrad_est.predict(X_test, random_state=random_seed)
metricframe_reductions = MetricFrame(
    metrics=metrics_dict,
    y_true=Y_test,
    y_pred=Y_pred_reductions,
    sensitive_features=A_test
)
metricframe_reductions.by_group

In [270]:
metricframe_reductions.by_group.to_csv('reductions_female.csv')

In [ ]:
pd.concat([metricframe_unmitigated.by_group,
           metricframe_postprocess.by_group,
          metricframe_reductions.by_group],
           keys=['Unmitigated', 'ThresholdOptimizer','Reductions'],
           axis=1)

In [ ]:
# Evaluate the difference between the largest and smallest value of each metric
metricframe_reductions.difference()

In [ ]:
pd.concat([metricframe_unmitigated.difference(),
           metricframe_postprocess.difference(),
           mf_deterministic.difference(),
          thresop_metricframe.difference(),
          mf_deterministic2.difference(),
          metricframe_reductions.difference()],
          keys=['Unmitigated: difference', 'ThresholdOptimizer: difference', 'DererministicThresholdOptimizer: Difference', 'ThresholdOptimizer_fprParity: Difference',
               'DeterministicThresholdOptimizer_fprParity: Difference','Reductions: Difference'],
          axis=1).T

In [237]:
pd.concat([metricframe_unmitigated.difference(),
           metricframe_postprocess.difference(),
           mf_deterministic.difference(),
          thresop_metricframe.difference(),
          mf_deterministic2.difference(),
          metricframe_reductions.difference()],
          keys=['Unmitigated: difference', 'ThresholdOptimizer: difference', 'DererministicThresholdOptimizer: Difference', 'ThresholdOptimizer_fprParity: Difference',
               'DeterministicThresholdOptimizer_fprParity: Difference','Reductions: Difference'],
          axis=1).T.to_csv("PerformanceComp_differenceFemale.csv")

While there is a decrease in the false negative rate difference from the unmitigated model, this decrease is not as substantial as with `ThresholdOptimizer`. Note, however, that `ThresholdOptimizer` was able to use the sensitive feature (i.e., race) at deployment time.

### Explore individual predictors

During the training process, the `ExponentiatedGradient` algorithm iteratively trains multiple inner models on a reweighted training dataset. The algorithm stores each of these predictors and then randomizes among them at deployment time.

In many applications, the randomization is undesirable, and also using multiple inner models can pose issues for interpretability. However, the inner models that `ExponentiatedGradient` relies on span a variety of fairness-accuracy trade-offs, and they could be considered for stand-alone deployment: addressing the randomization and interpretability issues, while possibly offering additional flexibility thanks to a variety of trade-offs. 

In this section explore the performance of the individual predictors learned by the `ExponentiatedGradient` algorithm. First, note that since the base estimator was `RandomForest` all these predictors are different RandomForest models:

In [ ]:
predictors = expgrad_est.predictors_
predictors

In [275]:
# Collect predictions by all predictors and calculate balanced error
# as well as the false negative difference for all of them

sweep_preds = [clf.predict(X_test) for clf in predictors]
balanced_error_sweep = [1-balanced_accuracy_score(Y_test, Y_sweep) for Y_sweep in sweep_preds]
fnr_diff_sweep = [false_negative_rate_difference(Y_test, Y_sweep, sensitive_features=A_test) for Y_sweep in sweep_preds]

In [ ]:
fnr_diff_sweep

In [ ]:
balanced_error_sweep

In [ ]:
# Show the balanced error / fnr difference values of all predictors on a raster plot  

plt.scatter(balanced_error_sweep, fnr_diff_sweep, label="ExponentiatedGradient - Iterations")
for i in range(len(predictors)):
  plt.annotate(str(i), xy=(balanced_error_sweep[i]+0.001, fnr_diff_sweep[i]+0.001))

# Also include in the plot the combined ExponentiatedGradient model
# as well as the three previously fitted models

plt.scatter(1-balanced_accuracy_score(Y_test, Y_pred_reductions),
            false_negative_rate_difference(Y_test, Y_pred_reductions, sensitive_features=A_test),
            label="ExponentiatedGradient - Combined model")
plt.scatter(1-balanced_accuracy_score(Y_test, Y_pred),
            false_negative_rate_difference(Y_test, Y_pred, sensitive_features=A_test),
            label="Unmitigated")
plt.scatter(1-balanced_accuracy_score(Y_test, Y_pred_postprocess),
            false_negative_rate_difference(Y_test, Y_pred_postprocess, sensitive_features=A_test),
            label="ThresholdOptimizer")
plt.scatter(1-balanced_accuracy_score(Y_test, y_pred_postprocess_deterministic),
            false_negative_rate_difference(Y_test, y_pred_postprocess_deterministic, sensitive_features=A_test),
            label="ThresholdOptimizer (DET)")

plt.xlabel("Balanced Error Rate")
plt.ylabel("False Negative Rate Difference")
plt.legend(bbox_to_anchor=(1.9,1))
plt.show()

## Comparing performance of different techniques

Now we have covered two different class of techniques for mitigating the fairness-related harms we found in our fairness-unaware model. In this section, we will compare the performance of the models we trained above across our key metrics.

#### Model performance - by group

In [243]:
def plot_technique_comparison(mf_dict, metric):
  """
  Plots a specified metric for a given dictionary of MetricFrames.
  """
  mf_dict = {k:v.by_group[metric] for (k,v) in mf_dict.items()}
  comparison_df = pd.DataFrame.from_dict(mf_dict)
  comparison_df.plot.bar(figsize=(12, 6), legend=False)
  plt.title(metric)
  plt.xticks(rotation=0, ha='center');
  plt.legend(bbox_to_anchor=(1.01,1), loc='upper left')

In [244]:
test_dict = {
    "Reductions": metricframe_reductions,
    "Unmitigated": metricframe_unmitigated,
    "Postprocessing": metricframe_postprocess,
    "Postprocessing (DET)": mf_deterministic
}

In [ ]:
plot_technique_comparison(test_dict, "false_negative_rate")

In [ ]:
plot_technique_comparison(test_dict, "balanced_accuracy")

In [ ]:
plot_technique_comparison(test_dict, "selection_rate")



#### Model performance - overall

In [278]:
overall_df = pd.DataFrame.from_dict({
    "Unmitigated": metricframe_unmitigated.overall,
    "Postprocessing": metricframe_postprocess.overall,
    "Postprocessing (DET)": mf_deterministic.overall,
    "Reductions": metricframe_reductions.overall
})

In [281]:
overall_df.T.to_csv('overall_female.csv')

In [ ]:
overall_df.transpose().plot.bar(subplots=True, layout= [1,4], figsize=(17, 5), legend=False, rot=-45, position=1.5);

In [282]:
difference_df = pd.DataFrame.from_dict({
    "Unmitigated": metricframe_unmitigated.difference(),
    "Postprocessing": metricframe_postprocess.difference(),
    "Postprocessing (DET)": mf_deterministic.difference(),
    "Reductions": metricframe_reductions.difference()
}
)

In [283]:
difference_df.T.to_csv('difference_female.csv')

In [ ]:
difference_df.T.plot.bar(subplots=True, layout= [1,4], figsize=(17, 5), legend=False, rot=-45, position=1.5);

### Randomized predictions

Both the `ExponentiatedGradient` and the `ThresholdOptimizer` yield randomized predictions (may return different result given the same instance). Due to legal regulations or other concerns, a practitioner may not be able to deploy a randomized model. To address these restrictions:

*   We created a deterministic predictor based on the randomized thresholds learned by the `ThresholdOptimizer`. This deteministic predictor achieved similar performance as the `ThresholdOptimizer`.
*   For the `ExponentiatedGradient` model, we can deploy one of the deterministic inner models rather than the overall `ExponentiatedGradient` model.



### Access to sensitive features





*   The `ThresholdOptimizer` model requires access to the sensitive features during BOTH training time and once deployed. If you do not have access to the sensitive features once the model is deployed, you will not be able to use the `ThresholdOptimizer`.
*   The `ExponentiatedGradient` model requires access to the sensitive features ONLY during training time. 




In [498]:
df_female.to_csv("dfML_female.csv")

In [ ]:
df_female.LOSClass.dtype

## USING CROSS VALIDATION FOR RANDOM FOREST CLASSIFIER

In [35]:
scoring=['precision_macro', 'recall_macro', 'balanced_accuracy']
rfmodel= Pipeline(steps=[
    ("preprocessing", StandardScaler()),
    ("Random_Forest", RandomForestClassifier())
])
scores=cross_validate(rfmodel,X,Y,scoring=scoring,cv=10)

In [36]:
sorted(scores.keys())

['fit_time',
 'score_time',
 'test_balanced_accuracy',
 'test_precision_macro',
 'test_recall_macro']

In [ ]:
scores['test_recall_macro'].mean()

In [ ]:
scores['test_balanced_accuracy'].mean()

In [ ]:
scores['test_precision_macro'].mean()

In [461]:
unmitigated_pipeline = Pipeline(steps=[
    ("preprocessing", StandardScaler()),
    ("Random_Forest", RandomForestClassifier())
])

FNR=[]
FPR=[]
Bal_Acc=[]
ROC=[]

for i in range(10):
    X_train_grp, X_test_grp, Y_train_grp, Y_test_grp, A_train_grp, A_test_grp, df_train_grp, df_test_grp = train_test_split(
        X,
        Y,
        A,
        df_ml,
        test_size=0.50,
        stratify=Y,
        random_state=None)
    X_train_bal_grp, Y_train_bal_grp, A_train_bal_grp = resample_dataset(X_train_grp, Y_train_grp, A_train_grp)
    
    unmitigated_pipeline.fit(X_train_bal_grp, Y_train_bal_grp)
    Y_pred_proba_grp = unmitigated_pipeline.predict_proba(X_test_grp)[:,1]
    Y_pred_grp = unmitigated_pipeline.predict(X_test_grp)
    FNR.append(false_negative_rate(Y_test_grp, Y_pred_grp))
    FPR.append(false_positive_rate(Y_test_grp, Y_pred_grp))
    Bal_Acc.append(balanced_accuracy_score(Y_test_grp, Y_pred_grp))
    ROC.append(roc_auc_score(Y_test_grp, Y_pred_proba_grp))

In [ ]:
accuracy_score(Y_test_grp, Y_pred_grp)==balanced_accuracy_score(Y_test_grp, Y_pred_grp)

In [463]:
FPR_MEAN, FPR_STD =np.array(FPR).mean(), np.array(FPR).std()
FNR_MEAN, FNR_STD =np.array(FNR).mean(), np.array(FNR).std()
Bal_Acc_MEAN, Bal_Acc_STD =np.array(Bal_Acc).mean(), np.array(Bal_Acc).std()
ROC_MEAN, ROC_STD =np.array(ROC).mean(), np.array(ROC).std()

In [ ]:
FPR_MEAN, FPR_STD

In [ ]:
FNR_MEAN, FNR_STD

In [ ]:
ROC_MEAN, ROC_STD

In [ ]:
Bal_Acc_MEAN, Bal_Acc_STD